__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x38_d03_reorganisieren.ipynb)__

# Reorganisieren und Reinigen
* speichert in data_03
* lex_test level 3
* 41 min

## ToDo
* TODO_verwaiste_lemmas
* TODO_TAGS_IN_MERKMALE


* löschliste am Ende nochmal anwenden
* Prüfen: ändert sich am score in translate_lex_full noch was, wenn echte Statistikzahlen da mit einfließen? Es geht um die Zuordnung lex --> lemma bei gegebenem tagZZ, also z.B. bei 'Alben' (--> Alb, Album, etc.).
* Symmetrie: Es gibt Lexeme für das Lemma "Ackermann/NN". Die gleichen Lexeme sollte es auch für das Lemma "Ackermann/NE" geben.
* Der gleiche Fall triff auch beim Lexem "ertragene" auf. Das sollte nicht nur ein Lexem des Verbs sein, sondern auch ein Lexem des Adjektivs.
* Noch besser: Lemmas zusammenfassen und beide Tag-Felder nutzen

## Spacy Nachschlagen 
* Kann das Lemma stimmen? >> Nachschlagen in wiktionary_merkmal (hier gibt es alle lex-lemma-Kombinationen, z.B. bei "Alben")
* Wenn nicht fündig geworden: Nachschlagen in translate_lex (hier gibt es nur 1 Lemma pro tagZZ)
* Resultat: Lemma

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 12:39:31


In [2]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [3]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, swifter

except ImportError as e:
    !pip install pandasklar swifter
    import pandasklar as pak, bpyth as bpy, swifter
    
grid       = pak.grid
from functools import partial
gridt = partial(grid, backend='dtale')    # dTale-Nutzung erzwungen
check_mask = pak.check_mask     


## Einstellungen

In [4]:
# Einstellungen

# Was debuggen?
suche_debug = ['besonnte','überlegte',]   
suche_debug = ['besonne',]  # auch gut: besinnen
suche_debug = ['inner-','innere','inner']  
suche_debug = ['Citrone',]  
suche_debug = ['Dezember','December','Citrone',]  
suche_debug = ['Anna',]  
# 'einen','beigen','Montage',
# '24/7','Schriftwerk'
suche_debug = ['gefriergetrocknet','gefriertrocknen','missraten']  
suche_debug = ['calvinistisch','kalvinistisch',]  


gründlich = True # einige Arbeitsschritte können weggelassen werden, wenn gründlich = True

# verbose
pak.Config.set('VERBOSE', True)

# Darstellung der Dataframes
pak.Config.set('GRID_BACKEND', '')   # standardmäßig kein dTale
pd.set_option('max_colwidth', 2000)  # aber dafür breiter

# Load
verzeichnis_load                       = 'data_02'
wiktionary_lemma_filename              = verzeichnis_load + '/wiktionary_lemma.pickle'  
translate_lex_filename                 = verzeichnis_load + '/wiktionary_nolemma.pickle'    # hieß vorher wiktionary_nolemma
wiktionary_merkmal_filename            = verzeichnis_load + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename       = verzeichnis_load + '/wiktionary_merkmal_text.pickle'  

# Steuertabellen
lex_test_filename                      = 'Steuertabellen/lex_test.xlsx'  
löschliste_filename                    = 'Steuertabellen/löschliste.xlsx'   
lexeme_manuell_filename                = 'Steuertabellen/lexeme_manuell.xlsx'                              # manuell eingepflegte Lexeme 
     
# Save
verzeichnis_save                       = 'data_03'
wiktionary_lemma_filename_save         = verzeichnis_save + '/wiktionary_lemma.pickle'  
translate_lex_filename_save            = verzeichnis_save + '/translate_lex.pickle'  
translate_lex_trash_filename_save      = verzeichnis_save + '/translate_lex_trash.pickle'  
translate_lemma_filename_save          = verzeichnis_save + '/translate_lemma.pickle'  
wiktionary_merkmal_filename_save       = verzeichnis_save + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename_save  = verzeichnis_save + '/wiktionary_merkmal_text.pickle'  



VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

GRID_BACKEND = 
--> setting backend= as default for all pandasklar functions



## Vorbereiten

In [5]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

In [6]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [7]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

In [8]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen import *
from x32_Funktionen                    import *

## Daten laden
* Steuertabellen
* wiktionary_lemma
* translate_lex
* wiktionary_merkmal     
* wiktionary_merkmal_text 
* jeweils einen Trash generieren

In [9]:
#blab.help(plan_merkmal_erstellen)

In [10]:
# plan_merkmal Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal, rows=9, column_definitions={ 'collect': {'maxWidth': 50,}, ' ': {'maxWidth': 50,}                   },     )

103 rows


,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
...,...,...,...,...,...,...
98,Superlativ,,,3,x3 Superlativ,True
99,Stamm,,,3,z1 Stamm,False
100,ipa,,,3,z1 ipa,False
101,noFlex,,E,3,z1 noFlex,True


In [11]:
a= pak.analyse_freqs(plan_merkmal,['plan','merkmal'])
grid(a, width=200)

3 rows


,plan,plan_count,plan_percent,merkmal,merkmal_count
0,3,59,57.3,"[Befehl du, Befehl ihr, Gerundivum, PTKVZ, VVIZU, lemma, lex, lexAlt, rhymes, Genus, Nom Sg, Nom Sg f, Nom Sg m, Nom Sg n, Gen Sg, Gen Sg f, Gen Sg m, Gen Sg n, Dat Sg, Dat Sg f]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,1,38,36.9,"[übersicht, noSteig, noSing, nurWortart, istName, Wikispecies, Wikivoyage, Taxonomie, Präfix, Suffix, VMaßeinheiten, def, herkunft, syn, lateral, gegen, klein, up, up_part, down]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,2,6,5.8,"[def, herkunft, bsp, bsp_re, bsp_gw, bsp_sp]","[1, 1, 1, 1, 1, 1]"


In [12]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3)
grid(lex_test)

166 rows loaded
Delete 3 rows from 166
163 rows


,level,lex,lemma,lex_tag_soll,notiz,check_tag
0,3,modern,modern,ADJA,kein Partizip,
1,3,auserlesen,auserlesen,ADJA,Partizip,
2,3,auserlesen,auslesen,VVPP,Partizip,
3,3,ausgerutscht,ausgerutscht,ADJA,Partizip,
4,3,ausgerutscht,ausrutschen,VVPP,Partizip,
...,...,...,...,...,...,...
161,1,naheliegend,naheliegen,VVFIN,Partizip (geraten),
162,3,erlesen,erlesen,ADJA,Partizip (Partizipialadjektiv fehlte),
163,1,erlesen,erlesen,VVFIN,Partizip (Partizipialadjektiv fehlte),
164,3,beschlagen,beschlagen,ADJA,Partizip (Partizipialadjektiv fehlte),


In [13]:
# lexeme_manuell laden
lexeme_manuell = pak.load_excel( lexeme_manuell_filename, tabcol='tab' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag',    'lemma_tag' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag_lex','lex_tag' )
lexeme_manuell = pak.move_cols(  lexeme_manuell, 'lex_tag','lemma_tag')
pak.sample(lexeme_manuell)

1794 rows loaded


,lex,lemma,lemma_tag,lex_tag,member,genus,tab
0,abberufen,abberufen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
164,gewußt,wissen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
304,überzeugend,überzeugen,VVFIN,Ptz1,<NA>,<NA>,Partizipien
487,Kids,Kid,NN,<NA>,<NA>,n,Diverses
489,Kontraindikationen,Kontraindikation,NN,<NA>,<NA>,f,Diverses
530,vergleiche,vergleichen,VVFIN,VVIMP,<NA>,<NA>,Diverses
561,zu viele,zu viel,ADV,ADV,<NA>,<NA>,Diverses
565,<NA>,24/7,ADJA,<NA>,<NA>,<NA>,Diverses
585,<NA>,bpm,NN,<NA>,Einheit,<NA>,Diverses
721,❀,_,$(,<NA>,<NA>,<NA>,Symbole


In [14]:
# Laden     
wiktionary_lemma         = pak.load_pickle(wiktionary_lemma_filename)
translate_lex            = pak.load_pickle(translate_lex_filename)
wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

171220 rows loaded
667548 rows loaded
4838905 rows loaded
657508 rows loaded


In [15]:
# rename_col
wiktionary_merkmal_text = pak.rename_col(wiktionary_merkmal_text,'section_id2','lemma_id')
translate_lex           = pak.rename_col(translate_lex,'lex','data')

In [16]:
# trash erzeugen     
wiktionary_lemma_trash        = wiktionary_lemma.head(0)
wiktionary_lemma_trash        = pak.write_empty_col(wiktionary_lemma_trash, 'msg', 'string')

translate_lex_trash           = translate_lex.head(0)
translate_lex_trash           = pak.write_empty_col(translate_lex_trash, 'msg', 'string')

wiktionary_merkmal_trash      = wiktionary_merkmal.head(0)
wiktionary_merkmal_trash      = pak.write_empty_col(wiktionary_merkmal_trash, 'msg', 'string')

wiktionary_merkmal_text_trash = wiktionary_merkmal_text.head(0)
wiktionary_merkmal_text_trash = pak.write_empty_col(wiktionary_merkmal_text_trash, 'msg', 'string')

## suche_debug

In [17]:
# wiktionary_lemma suche_debug
grid(search_str(wiktionary_lemma, suche_debug))

1 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score
122752,calvinistisch,572869,0,calvinistisch,calvinistisch,False,1,ADJA,ADJA,12sEJ7bQ43,ADJ,A,,,43.829163


In [18]:
# wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma_trash[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score,msg


In [19]:
pak.analyse_freqs(wiktionary_lemma_trash,['msg','lemma_id'])

'No rows'

In [20]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

19 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,score,lemma_id,lemma,lemma_lower,lemma_tagZZ
281025,calvinistischer,calvinistischer,LEX_A,LEX_A,lgw2hnfcZO,,,Deklinierte_Form,1.340219,calvinistisch,calvinistisch,calvinistisch,A
281026,calvinistischer_1,calvinistischer,LEX_A,LEX_A,INGP8W2kSo,,,Komparativ Adjektiv,0.714286,calvinistisch,calvinistisch,calvinistisch,A
281027,calvinistischste,calvinistischste,LEX_A,LEX_A,jsmBpF75kG,,,Deklinierte_Form,1.936817,calvinistisch,calvinistisch,calvinistisch,A
281028,calvinistischstes,calvinistischstes,LEX_A,LEX_A,pciCr0S6yo,,,Deklinierte_Form,1.264885,calvinistisch,calvinistisch,calvinistisch,A
281029,calvinistischster,calvinistischster,LEX_A,LEX_A,GfkXaeuibx,,,Deklinierte_Form,1.393682,calvinistisch,calvinistisch,calvinistisch,A
281030,calvinistischsten,calvinistischsten,LEX_A,LEX_A,MjTbURC5Hq,,,Deklinierte_Form,2.281895,calvinistisch,calvinistisch,calvinistisch,A
281031,calvinistischsten_1,calvinistischsten,LEX_A,LEX_A,aRysRy1rJL,,,Superlativ Adjektiv,0.765306,calvinistisch,calvinistisch,calvinistisch,A
281032,calvinistischstem,calvinistischstem,LEX_A,LEX_A,E9MhwEUhUa,,,Deklinierte_Form,0.991837,calvinistisch,calvinistisch,calvinistisch,A
281033,calvinistischere,calvinistischere,LEX_A,LEX_A,3qCZsPVlKI,,,Deklinierte_Form,1.938032,calvinistisch,calvinistisch,calvinistisch,A
281034,calvinistischeres,calvinistischeres,LEX_A,LEX_A,8RQSTrdhKW,,,Deklinierte_Form,1.266100,calvinistisch,calvinistisch,calvinistisch,A


In [21]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma_id == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

23 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
3161411,calvinistisch,Komparativ,calvinistischer,calvinistischer,,ADJA,,,7.00486,True
3145845,calvinistisch,Positiv,calvinistisch,calvinistisch,,ADJA,,,7.00486,True
3179604,calvinistisch,Superlativ,calvinistischsten,calvinistischsten,,ADJA,,,7.00486,True
1069863,calvinistisch,alt,calvinistisch~kalvinistisch,kalvinistisch,,ADJA,,,7.00486,True
1069847,calvinistisch,def,Calvinismus,Calvinismus,,ADJA,1,,7.00486,False
1069850,calvinistisch,gegen,buddhistisch,buddhistisch,,ADJA,1,,7.00486,False
1069851,calvinistisch,gegen,hinduistisch,hinduistisch,,ADJA,1,,7.00486,False
1069852,calvinistisch,gegen,islamisch,islamisch,,ADJA,1,,7.00486,False
1069853,calvinistisch,gegen,jüdisch,jüdisch,,ADJA,1,,7.00486,False
1069854,calvinistisch,gegen,katholisch,katholisch,,ADJA,1,,7.00486,False


In [22]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(suche_debug) #== 'gefriergetrocknet'
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

26 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
3145845,calvinistisch,Positiv,calvinistisch,calvinistisch,,ADJA,,,7.004860,True
1069863,calvinistisch,alt,calvinistisch~kalvinistisch,kalvinistisch,,ADJA,,,7.004860,True
263354,Hugenotte,def,calvinistisch,calvinistisch,,NN,1,Historie,12.213852,False
906929,Calvinist,familie,calvinistisch,calvinistisch,,NN,,,7.314703,False
362629,Calvinismus,familie_1,calvinistisch,calvinistisch,,NN,,,8.583232,False
1069891,calvinistische,gm,calvinistisch,calvinistisch,,,,,1.891859,False
1069897,calvinistischem,gm,calvinistisch,calvinistisch,,,,,0.959184,False
1069895,calvinistischen,gm,calvinistisch,calvinistisch,,,,,2.283111,False
1069865,calvinistischer,gm,calvinistisch,calvinistisch,,,,,1.340219,False
1069867,calvinistischer_1,gm,calvinistisch,calvinistisch,,,,,0.714286,False


In [23]:
# wiktionary_merkmal
# data_id ist bei allen gm-Merkmalen potentiell falsch!
mask1 = wiktionary_merkmal.data == 'besonnen'
mask2 = wiktionary_merkmal.merkmal == 'gm'
mask = mask1  &  mask2
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

29 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
1526920,besonn,gm,besonnen,besonnen,,,,,0.930612,False
1526922,besonne,gm,besonnen,besonnen,,,,,1.145808,False
1526928,besonnend,gm,besonnen,besonnen,,ADJA,,,0.459184,False
716611,besonnene,gm,besonnen,besonnen,,,,,1.741191,False
716606,besonnenem,gm,besonnen,besonnen,,,,,0.775510,False
716607,besonnenen,gm,besonnen,besonnen,,,,,2.168894,False
716608,besonnener,gm,besonnen,besonnen,,,,,1.212637,False
716609,besonnener_1,gm,besonnen,besonnen,,,,,0.540816,False
716616,besonnenere,gm,besonnen,besonnen,,,,,1.741191,False
716612,besonnenerem,gm,besonnen,besonnen,,,,,0.730612,False


In [24]:
# Genügend Datensätze?
print(wiktionary_merkmal.shape[0])
assert wiktionary_merkmal.shape[0] > 4800 * 1000

4838905


In [25]:
# Prüfen: is_unique
assert wiktionary_lemma.lemma_id.is_unique
assert translate_lex.nolemma_id.is_unique

## wiktionary_merkmal
* is_lex

In [26]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [27]:
# korrigieren: Punkt hinten ergänzen
startet_nicht_mit = ( '.', 'Industrie', 'St.')
endet_nicht_mit   = ( '.', 'Tag', 'Ort','Länge','Sieg')
mask1 =  wiktionary_merkmal.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal.data.str.contains('[0123456789]', regex=True)  
mask3 = ~wiktionary_merkmal.data.str.startswith(startet_nicht_mit)
mask4 = ~wiktionary_merkmal.data.str.endswith(  endet_nicht_mit)
mask5 = (wiktionary_merkmal.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask6 =  wiktionary_merkmal.data.str.len() > 2
mask7 = ~wiktionary_merkmal.merkmal.isin(['ipa'])
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask( wiktionary_merkmal, mask, 233 )

wiktionary_merkmal.loc[mask,'data'] += '.'
pak.sample(wiktionary_merkmal[mask])

check_mask:  241 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
5374,Straßenbahn,abk,Straßenbahn~Str.-B,Str.-B.,,NN,1,,25.515188,True
64801,a._D.,alt_2,a._D.~a. d. D,a. d. D.,,,3,,0.000000,True
78720,p._A.,alt_2,p._A.~p. Adr,p. Adr.,,TODO,1,,1.054678,True
101860,Ingenieur,abk,Ingenieur~Dipl.-Ing,Dipl.-Ing.,,NN,1,,28.157959,True
102378,Geburtstag,abk,Geburtstag~Geb.-T,Geb.-T.,,NN,"1,2",,106.701096,True
141173,BfA,up,,K. d. ö. R.,,NN,1,,3.417983,False
281658,zur_Zeit,abk,zur_Zeit~z. Zt,z. Zt.,,KOMBI,1,,8.736330,True
381098,östliche_Länge,abk,östliche_Länge~östl. L,östl. L.,,NN,,,2.668287,True
435149,Reserve,abk,Reserve~d. R,d. R.,,NN,4,,32.885784,True
586750,Mag.,weib,Mag.a,Mag.a.,,TODO,1,,7.693803,True


In [28]:
#wiktionary_merkmal.head(1)

In [29]:
# Merkmale, die eigentlich tags sind
mask = wiktionary_merkmal.merkmal.isin(alle_tags)
check_mask( wiktionary_merkmal, mask, 10 )

wiktionary_merkmal.loc[mask,'lemma_tag'] = ''
wiktionary_merkmal.loc[mask,'data_tag']  = wiktionary_merkmal[mask].merkmal
wiktionary_merkmal.loc[mask,'merkmal']   = 'lex'
wiktionary_merkmal.loc[mask,'is_lex']    = True
grid(wiktionary_merkmal,mask)

check_mask:  10 rows
10 rows out of 4838905


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
1830808,zu_viel,lex,zu_viele,zu viele,ADV,,,manuell,8.736330,True
1830811,vergleichen,lex,vergleiche,vergleiche,VVIMP,,,manuell,14.083839,True
1830812,herumpimpeln_M,lex,herumzupimpeln,herumzupimpeln,VVIZU,,,manuell,8.736330,True
1830813,anstaunen,lex,anzustaunen,anzustaunen,VVIZU,,,manuell,8.736330,True
1830814,einreden,lex,einzureden,einzureden,VVIZU,,,manuell,9.020657,True
1830815,angucken_M,lex,anzugucken,anzugucken,VVIZU,,,manuell,8.736330,True
1830816,loswerden,lex,loszuwerden,loszuwerden,VVIZU,,,manuell,43.147022,True
1830817,herauslassen_M,lex,herauslassen_M~herauszulassen,herauszulassen,VVIZU,,,manuell,8.736330,True
1830818,aufrütteln_M,lex,aufzurütteln,aufzurütteln,VVIZU,,,manuell,8.736330,True
1830819,hereinlassen_M,lex,hereinlassen_M~hereinzulassen,hereinzulassen,VVIZU,,,manuell,8.736330,True


In [30]:
# Mehrfache Spaces >> Einfacher Space (eigentlich hätten die alle schon von Preprocess erwischt werden müssen)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

In [31]:
# Mehrfachbindestrich
mask =  wiktionary_merkmal.data == '--'
wiktionary_merkmal.loc[mask,'data'] = '-'

In [32]:
# Hochkommas entfernen
mask1 = wiktionary_merkmal.data.str.count("\'") > 1
mask2 = wiktionary_merkmal.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal,mask, 0, 20)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.replace("'", '')

check_mask:  7 rows


In [33]:
# Leerstring data Müll löschen
mask = wiktionary_merkmal['data'] == ''
check_mask(wiktionary_merkmal,mask,0,20)

msg = 'Leerstring data Müll'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
grid(wiktionary_merkmal_trash, msg)

check_mask:  6 rows
Move 6 rows from 4838905
6 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex,msg
2185496,Covid,Genus,,,,NE,,,8.626974,False,Leerstring data Müll
2186567,Reiwa,Genus,,,,NN,,,7.551641,False,Leerstring data Müll
2186574,Heisei,Genus,,,,NN,,,7.168894,False,Leerstring data Müll
2187714,Außerm_Pfarrgarten,Genus,,,,NE,,,2.459295,False,Leerstring data Müll
2201977,Am_Hauptbahnhof,Genus,,,,NE,,,3.346294,False,Leerstring data Müll
2201997,Am_Bahnhof,Genus,,,,NE,,,2.467801,False,Leerstring data Müll


In [34]:
# fillna
wiktionary_merkmal['meta']       = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']   = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']  = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']        = wiktionary_merkmal.num.fillna('')

## translate_lex ==========
* hieß bislang wiktionary_nolemma
* Gerundivum als Merkmal speichern
* verwaiste löschen
* Spalten umbenennen
* pflege_translate_tabelle

In [35]:
assert not pak.any_nan(translate_lex)

In [36]:
# lemma_id vorhanden?
mask = translate_lex.lemma_id == ''
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [37]:
# nolemma_id vorhanden und nicht widersprüchlich?
mask = translate_lex.nolemma_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='nolemma_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


### Gerundivum als Merkmal speichern

In [38]:
# Gerundivum finden
mask1 =  translate_lex.data.str.contains('zu')
mask2 = ~translate_lex.data.str.endswith('zu')
mask3 =  translate_lex.tag.str.contains('LEX_A') 
mask4 =  translate_lex.member.str.contains('Dekliniertes_Gerundivum') 
mask = mask1  & mask2  &  mask3  &  mask4
check_mask(translate_lex,mask,16480)

df = translate_lex[mask].copy()
pak.sample(df)

check_mask:  16480 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,score,lemma_id,lemma,lemma_lower,lemma_tagZZ
359940,aufzuwärmende,aufzuwärmende,LEX_A,LEX_A,TIJcHNZnn0,,,Dekliniertes_Gerundivum Adjektiv,1.763062,aufwärmen,aufwärmen,aufwärmen,V
364155,abzustrafendem,abzustrafendem,LEX_A,LEX_A,3XmuYPtJvS,,,Dekliniertes_Gerundivum Adjektiv,0.812245,abstrafen,abstrafen,abstrafen,V
409567,abzuarbeitende,abzuarbeitende,LEX_A,LEX_A,sSydwL7m9s,,,Dekliniertes_Gerundivum Adjektiv,1.749696,abarbeiten,abarbeiten,abarbeiten,V
409791,abzubedingende,abzubedingende,LEX_A,LEX_A,a98HNpOefT,,,Dekliniertes_Gerundivum Adjektiv,1.746051,abbedingen,abbedingen,abbedingen,V
410197,abzubrühende,abzubrühende,LEX_A,LEX_A,fJpqpPYTyL,,,Dekliniertes_Gerundivum Adjektiv,1.712029,abbrühen,abbrühen,abbrühen,V
443749,hinzuzufindendem,hinzuzufindendem,LEX_A,LEX_A,3vlHJjo7pW,,,Dekliniertes_Gerundivum Adjektiv,0.757143,hinzufinden,hinzufinden,hinzufinden,V
483133,zusammenzubindenden,zusammenzubindenden,LEX_A,LEX_A,00jTsBOdYD,,,Dekliniertes_Gerundivum Adjektiv,2.191981,zusammenbinden,zusammenbinden,zusammenbinden,V
528620,hochzuschätzenden,hochzuschätzenden,LEX_A,LEX_A,zzooxDokpP,,,Dekliniertes_Gerundivum Adjektiv,2.145808,hochschätzen,hochschätzen,hochschätzen,V
544538,einzucheckendem,einzucheckendem,LEX_A,LEX_A,003ywklN2d,,,Dekliniertes_Gerundivum Adjektiv,0.897959,einchecken,einchecken,einchecken,V
560476,wertzuschätzenden,wertzuschätzenden,LEX_A,LEX_A,d2WiAXF6fZ,,,Dekliniertes_Gerundivum Adjektiv,2.405832,wertschätzen,wertschätzen,wertschätzen,V


In [39]:
# anfügen vorbereiten
df = pak.rename_col( df, 'lex',        'data' )
df = pak.rename_col( df, 'nolemma_id', 'data_id' )
df['merkmal']  = 'Gerundivum'
df['meta']     = ''
df['num']      = ''
df['is_lex']   = True
df['data_tag'] = 'ADJA'
df = pak.drop_cols( df, set(df.columns) - set(wiktionary_merkmal.columns) )
df = pak.update_col(df, wiktionary_lemma, left_on='lemma_id', right_on='lemma_id', col='tag', col_rename='lemma_tag')

update_col: 16480 cells written into new column


In [40]:
# Prüfen: Alle Gerundiva verweisen auf ein Verb
mask_L = wiktionary_lemma.tagZZ == 'V'
verben = wiktionary_lemma[mask_L].lemma_id

mask = ~df.lemma_id.isin(verben)
check_mask(df,mask,0)

check_mask:  0 rows


In [41]:
# anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df)

16480 rows added, now a total of 4855379


In [42]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [43]:
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
pak.sample(wiktionary_merkmal[mask])

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
4838899,aufwärmen,Gerundivum,aufzuwärmende,aufzuwärmende,ADJA,VVFIN,,,1.763062,True
4839077,anleuchten,Gerundivum,anzuleuchtenden,anzuleuchtenden,ADJA,VVFIN,,,2.105711,True
4840750,abarbeiten,Gerundivum,abzuarbeitende,abzuarbeitende,ADJA,VVFIN,,,1.749696,True
4840794,abberufen,Gerundivum,abzuberufende,abzuberufende,ADJA,VVFIN,,,1.731470,True
4844943,hinzufinden,Gerundivum,hinzuzufindendem,hinzuzufindendem,ADJA,VVFIN,,,0.757143,True
4852001,wertschätzen,Gerundivum,wertzuschätzenden,wertzuschätzenden,ADJA,VVFIN,,,2.405832,True
4853548,übriglassen,Gerundivum,übrigzulassendes,übrigzulassendes,ADJA,VVFIN,,,1.240583,True
4855378,entgegenwerfen,Gerundivum,entgegenzuwerfendes,entgegenzuwerfendes,ADJA,VVFIN,,,1.154313,True


### translate_lex pflegen

In [44]:
#grid(translate_lex,pak.sample)

In [45]:
# Verwaiste löschen
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,30)

msg = 'verwaist_lemma_id'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  9 rows
Move 9 rows from 667548
9 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,score,lemma_id,lemma,lemma_lower,lemma_tagZZ,msg
884,zagen_1,zagen,LEX_A,LEX_A,0CkESsY0SY,,,Deklinierte_Form,2.534629,zage,zag,zag,A,verwaist_lemma_id
28326,Kremes,Kremes,LEX_A,LEX_A,WJPRmiOLt6,,,Deklinierte_Form,0.742857,Cremes,Kreme,kreme,N,verwaist_lemma_id
177595,Meßgewande,Meßgewande,LEX_A,LEX_A,binOH2u11m,,,Deklinierte_Form,0.600000,Messgewande,Meßgewand,meßgewand,N,verwaist_lemma_id
204892,Zierates,Zierates,LEX_A,LEX_A,mQtZge3RLQ,,,Deklinierte_Form,0.469388,Zierats,Zierat,zierat,N,verwaist_lemma_id
204893,Zierats,Zierats,LEX_A,LEX_A,7WDIuMD3u6,,,Deklinierte_Form,0.465306,Zierates,Zierat,zierat,N,verwaist_lemma_id
281103,kalvinistischsten,kalvinistischsten,LEX_A,LEX_A,00ocFjdZ25,,,Deklinierte_Form,2.207777,calvinistischsten,kalvinistisch,kalvinistisch,A,verwaist_lemma_id
281104,kalvinistischsten_1,kalvinistischsten,LEX_A,LEX_A,FUTCY0ysRx,,,Superlativ Adjektiv,0.640816,calvinistischsten,kalvinistisch,kalvinistisch,A,verwaist_lemma_id
654680,Fischgratparkettes,Fischgratparkettes,LEX_A,LEX_A,Od1lUBM2SG,,,Deklinierte_Form,0.587755,Fischgratparketts,Fischgratparkett,fischgratparkett,N,verwaist_lemma_id
654681,Fischgratparketts,Fischgratparketts,LEX_A,LEX_A,yQIN27bNN1,,,Deklinierte_Form,1.066829,Fischgratparkettes,Fischgratparkett,fischgratparkett,N,verwaist_lemma_id


In [46]:
# umformen
translate_lex = pak.rename_col( translate_lex, 'lex',        'data' )
translate_lex = pak.rename_col( translate_lex, 'nolemma_id', 'data_id' )
translate_lex['member'] = ('wiki ' + translate_lex['member'] ).str.strip()
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma)

update_col: 667539 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 667539 cells written into existing column
update_col: 667539 cells written into new column
renaming score --> data_score


In [47]:
# Müll
mask = translate_lex.lemma_tag == 'TODO'
check_mask(translate_lex,mask,0,5)

msg = 'TODO'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )

check_mask:  5 rows
Move 5 rows from 667539


In [48]:
# not any nan
assert not pak.any_nan(translate_lex)

In [49]:
pak.sample(translate_lex)

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,lieben,,2.408262,wiki Deklinierte_Form,lieb,lieb,ADJA,A,21.805347,lieb
43,mich,mich,,0.795918,wiki Deklinierte_Form,ich,ich,PPER,P,5717.436035,ich
384,Marken_1,Marken,,0.746939,wiki Deklinierte_Form,Mark,Mark,NN,N,367.630859,mark
8285,aufzuwerfen,aufzuwerfen,,0.540816,wiki Erweiterter_Infinitiv Verb,aufwerfen,aufwerfen,VVFIN,V,75.192467,aufwerfen
54677,unterschlage,unterschlage,,1.000000,wiki Konjugierte_Form Verb,unterschlagen,unterschlagen,VVFIN,V,84.217979,unterschlagen
188180,überführte,überführte,,1.761847,wiki Deklinierte_Form,überführt_4,überführt,ADJA,A,0.160714,überführt
188314,α-Zerfällen,α-Zerfällen,,0.434694,wiki Deklinierte_Form,α-Zerfall,α-Zerfall,NN,N,119.873871,α-zerfall
194765,1-Cent-Stücks,1-Cent-Stücks,,0.706122,wiki Deklinierte_Form,1-Cent-Stück,1-Cent-Stück,NN,N,21.782990,1-cent-stück
362413,vermarktende,vermarktende,,1.749696,wiki Deklinierte_Form,vermarktend,vermarktend,ADJA,A,4.285714,vermarktend
434462,anzuwettern,anzuwettern,,0.457143,wiki Erweiterter_Infinitiv Verb,anwettern,anwettern,VVFIN,V,12.094533,anwettern


In [50]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

17 rows


,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
281020,calvinistischer,calvinistischer,,1.340219,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281021,calvinistischer_1,calvinistischer,,0.714286,wiki Komparativ Adjektiv,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281022,calvinistischste,calvinistischste,,1.936817,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281023,calvinistischstes,calvinistischstes,,1.264885,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281024,calvinistischster,calvinistischster,,1.393682,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281025,calvinistischsten,calvinistischsten,,2.281895,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281026,calvinistischsten_1,calvinistischsten,,0.765306,wiki Superlativ Adjektiv,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281027,calvinistischstem,calvinistischstem,,0.991837,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281028,calvinistischere,calvinistischere,,1.938032,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch
281029,calvinistischeres,calvinistischeres,,1.266100,wiki Deklinierte_Form,calvinistisch,calvinistisch,ADJA,A,43.829163,calvinistisch


In [51]:
# anschauen
#pak.analyse_freqs(translate_lex,['member','lemma_tag','lex'])

## Partizipien ==========
Für fast jedes Partizip gibt es zwei Datensätze: <br> 
1) in wiktionary_merkmal.data: Die Partizip-Flexion des Verbs. merkmal ist 'Ptz1' oder 'Ptz2'.<br> 
2) in wiktionary_lemma.lemma:  Als Adjektiv mit member 'Partizip'. Es gibt ca. 320 Ausnahmen, nicht alle Partizipien sind auch Adjektive.

In [52]:
# Falsche Partizipien löschen
falsche_partizipien = ['sehen','mögen','hören','sollen','können','lassen']
mask1 = wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask2 = wiktionary_merkmal.data.isin(falsche_partizipien)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0,6)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask, 'Falsche_Partizipien')
#wiktionary_merkmal[mask]

check_mask:  6 rows
Move 6 rows from 4855379


In [53]:
# verben_ptz1: Alle Ptz1-Merkmale der Verben
spalten = ['lemma_id','data_id','data','score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz1'
verben_ptz1 = wiktionary_merkmal[mask][spalten]
verben_ptz1 = pak.update_col(df_to=verben_ptz1, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz1 = pak.move_cols(       verben_ptz1,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz1, suche_debug)
if len(a) == 0:
    a = verben_ptz1.sample(5)
a

,lemma_id,lemma,data_id,data,score,merkmal
1827574,umherrutschen,umherrutschen,umherrutschend,umherrutschend,4.886999,Ptz1
1813651,aufsperren,aufsperren,aufsperrend,aufsperrend,11.207776,Ptz1
1808441,nachlassen,nachlassen,nachlassend,nachlassend,31.179831,Ptz1
1806004,bevormunden,bevormunden,bevormundend,bevormundend,18.527340,Ptz1
1814095,nachsagen,nachsagen,nachsagend,nachsagend,5.995140,Ptz1


In [54]:
# Kein leeres data_id
mask = verben_ptz1.data_id.isnull()  |  (verben_ptz1.data_id == '')
check_mask(verben_ptz1,mask,0)

check_mask:  0 rows


In [55]:
# alle enden mit 'nd'
mask = ~verben_ptz1.data.str.endswith('nd')
check_mask(verben_ptz1,mask,0)
#verben_ptz[mask]

check_mask:  0 rows


In [56]:
# verben_ptz2: Alle Ptz2-Merkmale der Verben
spalten = ['lemma_id','data_id','data','score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz2'
verben_ptz2 = wiktionary_merkmal[mask][spalten]
verben_ptz2 = pak.update_col(df_to=verben_ptz2, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz2 = pak.move_cols(       verben_ptz2,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz2, suche_debug)
if len(a) == 0:
    a = verben_ptz2.sample(5)
a

,lemma_id,lemma,data_id,data,score,merkmal
1808281,einatmen,einatmen,eingeatmet,eingeatmet,18.651276,Ptz2
1808054,säumen,säumen,gesäumt,gesäumt,16.394897,Ptz2
1808698,überführen,überführen,übergeführt,übergeführt,8.812880,Ptz2
3125239,ergraben_1,ergraben,ergraben,ergraben,8.736330,Ptz2
1822197,abspielen,abspielen,abgespielt,abgespielt,35.739975,Ptz2


In [57]:
# Kein leeres data_id
mask = verben_ptz2.data_id.isnull()  |  (verben_ptz2.data_id == '')
check_mask(verben_ptz2,mask,0)

check_mask:  0 rows


In [58]:
# adjektive_ptz1: Adjektive mit Ptz1-Markierung
spalten = ['lemma_id','lemma','score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz1')
mask = mask1  &  mask2 
adjektive_ptz1 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz1, suche_debug)
if len(a) == 0:
    a = adjektive_ptz1.sample(5)
a

,lemma_id,lemma,score,tag,member,data_id
125482,durchreisend_1,durchreisend,1.683673,ADJA,Ptz1,durchreisen
165825,widerredend,widerredend,4.726531,ADJA,Ptz1,widerreden
149274,pfarrend,pfarrend,4.493877,ADJA,Ptz1,pfarren
147671,niederfallend,niederfallend,4.395918,ADJA,Ptz1,niederfallen
149192,perlend,perlend,4.542857,ADJA,Ptz1,perlen_1


In [59]:
# Kein leeres data_id
# data_id verweist auf das Verb
mask = adjektive_ptz1.data_id.isnull()  |  (adjektive_ptz1.data_id == '')
check_mask(adjektive_ptz1,mask,0)

check_mask:  0 rows


In [60]:
# adjektive_ptz2: Adjektive mit Ptz2-Markierung
spalten = ['lemma_id','lemma','score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2 
adjektive_ptz2 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz2, suche_debug)
if len(a) == 0:
    a = adjektive_ptz2.sample(5)
a

,lemma_id,lemma,score,tag,member,data_id
116685,aufgehört,aufgehört,1.404082,ADJA,Ptz2,aufhören
111991,abecediert_1,abecediert,0.487755,ADJA,Ptz2,abecedieren
120937,beschottert,beschottert,1.559184,ADJA,Ptz2,beschottern
162783,veruntreut_1,veruntreut,0.420408,ADJA,Ptz2,veruntreuen
112249,abgefingert,abgefingert,1.420408,ADJA,Ptz2,abfingern


In [61]:
# Kein leeres data_id
# data_id verweist auf das Verb
# Das sind Kandidaten für die Aufnahme in lexeme_manuell, Reiter Partizipien
nöö = ['befangen','betreten','durchwachsen','erfahren','erlesen','erwachsen','wollen']
mask1 = adjektive_ptz2.data_id.isnull()  |  (adjektive_ptz2.data_id == '')
mask2 = ~adjektive_ptz2.lemma.isin(nöö)
mask = mask1  &  mask2
#grid(adjektive_ptz2,mask)
check_mask(adjektive_ptz2,mask,0)

check_mask:  0 rows


In [62]:
#for p in adjektive_ptz2[mask].lemma:
#    print(p)

In [63]:
# adjektive_noptz: Adjektive ohne Ptz-Markierung
spalten = ['lemma_id','lemma','score','tag','member','data_id']
mask1 =  wiktionary_lemma.tagZ == 'ADJ'
mask2 = ~wiktionary_lemma.member.str.contains('Ptz1')
mask3 = ~wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2  &  mask3  
adjektive_noptz = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_noptz, suche_debug)
if len(a) == 0:
    a = adjektive_noptz.sample(5)
a

,lemma_id,lemma,score,tag,member,data_id
122752,calvinistisch,calvinistisch,43.829163,ADJA,,


In [64]:
# Nur leeres data_id
mask = adjektive_noptz.data_id.isnull()  |  (adjektive_noptz.data_id == '')
#grid(adjektive_noptz,~mask)
check_mask(adjektive_noptz,~mask,0)

check_mask:  0 rows


In [65]:
# Das sind vermutlich eigentlich Partizipien 
# Partizip (geraten)
mask1 = adjektive_noptz.lemma.str.endswith('nd')
mask2 = ~adjektive_noptz.lemma.str.contains(' ')
mask = mask1  &  mask2
kandidaten = adjektive_noptz[mask].copy()

# Kandidaten aufbereiten
nöö = ['dutzend','flektierend','frappierend','spannend']
kandidaten['verb'] = kandidaten.lemma.str[:-1]
mask =  wiktionary_lemma.tagZZ == 'V'
kandidaten = pak.update_col(kandidaten, 
                            wiktionary_lemma[mask],
                            left_on=['verb'],
                            right_on=['lemma'],
                            col='lemma_id',
                            col_rename='data_id',
                            func='min'
                           )

# und filtern
maskA = kandidaten.data_id.isnull()  |  (kandidaten.data_id == '')
maskB = kandidaten.lemma.isin(nöö)
mask = maskA  |  maskB
kandidaten = pak.drop_rows(kandidaten,mask)

spalten = ['lemma','verb']
grid(kandidaten[spalten])

update_col: func min applied, 382 records less!
update_col: 8 cells written into existing column
Delete 297 rows from 301
4 rows


,lemma,verb
121264,bestimmend,bestimmen
129409,erregend,erregen
147265,naheliegend,naheliegen
147271,nahestehend,nahestehen


### Gibt es Verben ohne Partizip-Lexeme?
* ggf. Partizipen manuell nachtragen

In [66]:
# Verben ohne Ptz1
spalten = ['lemma_id','lemma','score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz1.lemma)
mask = mask1  &  mask2 
check_mask(wiktionary_lemma,mask,26)
verben_ohne_ptz1 = wiktionary_lemma[mask][spalten]
#verben_ohne_ptz1

check_mask:  33 rows


In [67]:
# Kandidaten für die manuelle Aufnahme
# - entweder als Ptz1-Lexem in lexeme_manuell (Reiter Partizipien)
# - oder als lemma in die löschliste

nöö = ['angedeihen','erstinken','gegenanargumentieren','grundsanieren','interdependieren','mittagessen','umfragen','urformen','voranmelden','zusammenreiten']
mask1 = verben_ohne_ptz1.lemma.str.endswith('en')
mask2 = ~verben_ohne_ptz1.lemma.isin(nöö)
mask = mask1  &  mask2
kandidaten = verben_ohne_ptz1[mask][['lemma']]
kandidaten = kandidaten.drop_duplicates().sort_values('lemma')
kandidaten['lex'] = kandidaten.lemma + 'd'
kandidaten = pak.move_cols(kandidaten,'lex')
kandidaten

,lex,lemma
115159,anguckend,angucken
132411,geborend,geboren
2100,handstoppend,handstoppen
145638,milbend,milben
146584,möchtend,möchten
154965,sprützend,sprützen
169161,zustande kommend,zustande kommen


In [68]:
# Verben ohne Ptz2
nöö = ['Schlange stehen', 'gehn', 'stehn', 'umsehn','vergleiche']
spalten = ['lemma_id','lemma','score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz2.lemma)
mask3 = ~wiktionary_lemma.lemma.isin(nöö)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,0,11)
verben_ohne_ptz2 = wiktionary_lemma[mask][spalten]
verben_ohne_ptz2

check_mask:  6 rows


,lemma_id,lemma,score,tag,member
115159,angucken_M,angucken,26.208990,VVFIN,manuell
117142,aufrütteln_M,aufrütteln,26.208990,VVFIN,manuell
132411,geboren_M,geboren,8.736330,VVFIN,manuell
138322,herumpimpeln_M,herumpimpeln,26.208990,VVFIN,manuell
146584,möchten,möchten,105.177872,VVFIN,
169161,zustande_kommen,zustande kommen,10.512030,VVFIN,Wortverbindung


In [69]:
del verben_ohne_ptz1
del verben_ohne_ptz2

In [70]:
#mask = wiktionary_merkmal.lemma_id == 'betreffen'
#wiktionary_merkmal[mask]

In [71]:
#mask = wiktionary_lemma.lemma == 'betreffend'
#grid(wiktionary_lemma[mask])

### Partizipien und Adjektive vergleichen

In [72]:
# merge für Ptz1
a = pd.merge(verben_ptz1, adjektive_ptz1, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))

In [73]:
# Übersicht
assert a.shape[0] > 13000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,14180,100.0,##################################################
1,left_only,0,0.0,
2,right_only,0,0.0,


In [74]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,20)
a[mask][['lemma_a']]

check_mask:  0 rows


,lemma_a


In [75]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,1)
#a[mask] 

check_mask:  0 rows


In [76]:
# merge für Ptz2
a = pd.merge(verben_ptz2, adjektive_ptz2, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))
a = a.drop_duplicates()

In [77]:
# Übersicht
assert a.shape[0] > 14000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,14197,98.9,#################################################
1,left_only,165,1.1,
2,right_only,0,0.0,


In [78]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,30)
a[mask][['lemma_id_a','lemma_a','tag','member']]

check_mask:  0 rows


,lemma_id_a,lemma_a,tag,member


In [79]:
# entkennzeichnen
entkennzeichnen = a[mask].lemma_id_a
mask_L = wiktionary_lemma.lemma_id.isin(entkennzeichnen)
wiktionary_lemma.loc[mask_L,'member'] = pak.remove_str(wiktionary_lemma[mask_L].member, 'Ptz2')
wiktionary_lemma[mask_L]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score


In [80]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,350)
fehlende_adjektive = a[mask]
fehlende_adjektive = pak.update_col(df_to=fehlende_adjektive, df_from=adjektive_noptz, left_on='lemma_v', right_on='lemma',col='lemma_id',col_rename='adj_id')
#fehlende_adjektive

check_mask:  165 rows
update_col: 7 cells written into new column


In [81]:
# Das Partizipialadjektiv fehlt. Das kann zwei Ursachen haben:
# (1) Es gibt ein Adjektiv, das ist aber nicht als Partizipialadjektiv markiert
# (2) Es gibt kein Adjektiv.
# Zuerst kümmern wir uns um Fall (1).
# Die in adj_id genannten Adjektive müssen noch als Partizipialadjektiv markiert werden
mask_a = ~fehlende_adjektive.adj_id.isnull()
check_mask(fehlende_adjektive,mask_a,0,20)
fehlende_adjektive[mask_a]

check_mask:  7 rows


,lemma_id_v,lemma_v,data_id_v,data,score_v,merkmal,lemma_id_a,lemma_a,score_a,tag,member,data_id_a,_merge,adj_id
14162,erlesen_1,erlesen,erlesen,erlesen,2.064399,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erlesen
14173,beschlagen_1,beschlagen,beschlagen,beschlagen,7.763062,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,beschlagen
14265,erwachsen_1,erwachsen,erwachsen,erwachsen,3.565006,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erwachsen
14291,betreten_1,betreten,betreten,betreten,4.420413,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,betreten
14305,befangen_1,befangen,befangen,befangen,3.855407,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,befangen
14306,durchwachsen_2,durchwachsen,durchwachsen,durchwachsen,1.496962,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,durchwachsen
14310,erfahren,erfahren,erfahren_1,erfahren,14.021872,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erfahren_1


In [82]:
# markieren
mask = wiktionary_lemma.lemma_id.isin(fehlende_adjektive[mask_a].adj_id)
check_mask(wiktionary_lemma,mask,0,20)
wiktionary_lemma.loc[mask,'member'] += ' Ptz2'
#wiktionary_lemma[mask]

check_mask:  7 rows


In [83]:
# Fall (2)
# Dies sind Kandidaten dafür, manuell aufgenommen zu werden.
# Und zwar als Partizipien (Ptz2) in lexeme_manuell, dann werden die Adjektive automatisch erstellt und passend verlinkt.
# Das geschieht in d01_tag_und_member.
#

mask_a = fehlende_adjektive.adj_id.isnull()
kandidaten = fehlende_adjektive[mask_a][['data']].drop_duplicates().sort_values('data')

nöö = ['abbegeben','abbehalten','abbekommen','abgetan','anbekommen','angetan','aufbekommen','aufgetan','ausgetan','begeben','behalten','beibehalten','bekommen',
       'benachrangt','bestoßen','betan','betragen','dabehalten','dargetan','dazugetan','dickgetan','dortbehalten',
       'durchbekommen','durchhauen','durchmarschiert','entraten','entschlagen','erfangen','ermessen','erschlafen','erschollen',
       'ersehen','ertragen','fallenlassen','fortwollen','ge- -zt','gegenanargumentiert','geheimgetan',
       'gemecht','genuggetan','geraidet','getan','gleichgetan','großgetan','gutgetan','hartgetan','hervorgetan','hinbekommen','hindurchmarschieren',
       'hineinbegeben','hineingetan','hinterlaufen','hinunterbekommen','hinzugetan','hinüberrannte','hängen lassen','leichtgetan','leidgetan',
       'lieb behalten','missfallen','mitbekommen','mitwollen','nachbekommen','nachgeraten','nachgetan','notgetan','schwergetan','schöngetan','stehenlassen',
       'umeinandergehetzt','umeinandergelegen','umeinandergestanden','umgefragt','umgetan','umtreten','unterbekommen','unterfangen','unterwaschen',
       'verhebt','verladen','verschleißt','versehen','vertan','vertragen',
       'vorgetan','weggetan','wegwollen','weh getan','wehgetan','widerfahren','widerraten','wiederwollen','wohlgetan','zubekommen','zugetan',
       'zugutegetan','zurechtbekommen','zurückbegeben','zurückbekommen','zurückerhalten','zurückgetan','zurückwollen','zusammenbekommen','zusammengetan','zuvorgetan',
       'überlassen','überschlafen','betroffen','geschehen','umschlagen']
print('len nöö =',len(nöö))
mask1 = ~kandidaten.data.str.startswith('-')
mask2 = ~kandidaten.data.isin(nöö)
mask = mask1  &  mask2  
check_mask(kandidaten,mask, 0,20)
grid(kandidaten,mask)

len nöö = 110
check_mask:  7 rows
7 rows out of 118


,data
14209,fortbegeben
14271,geadert
14184,gesprützt
14273,geädert
14163,herumbekommen
14322,photogeshoppt
14155,zusammengeraten


### Aufräumen

In [84]:
del verben_ptz1
del verben_ptz2
del adjektive_ptz1
del adjektive_ptz2
del adjektive_noptz

## wiktionary_lemma ==========

In [85]:
# lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

check_mask:  0 rows


In [86]:
# wiktionary_lemma score ergänzen
# Der score in translate_lex spiegelt die Länge des Wiktionary_Eintrags des Lexem wieder. Der längste wird zum Lemma draufaddiert.
wiktionary_lemma['score']     = wiktionary_lemma.score.fillna(0) 
translate_lex['data_score']   = translate_lex.data_score.fillna(0) 
wiktionary_lemma = pak.update_col(wiktionary_lemma, translate_lex, on=['lemma_id'], col='data_score', col_rename='score_add', func='max')
wiktionary_lemma['score_add'] = wiktionary_lemma.score_add.fillna(0) 

wiktionary_lemma['score']     = wiktionary_lemma.score  +  (wiktionary_lemma.score_add / 2.0)
wiktionary_lemma = pak.drop_cols(wiktionary_lemma, 'score_add')

update_col: func max applied, 525590 records less!
update_col: 141944 cells written into new column


In [87]:
#wiktionary_lemma

In [88]:
# Dups?
mask = wiktionary_lemma.duplicated(subset=['lemma_id'])
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows


In [89]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma['tag'] = tag_sortieren(wiktionary_lemma.tag)

In [90]:
# Spalten wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0,10) # ein paar können wir tolerieren

check_mask:  303 rows
check_mask:  0 rows


In [91]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score
122752,calvinistisch,572869,0,calvinistisch,calvinistisch,False,1,ADJA,ADJA,vxjkmPSKuu,ADJ,A,,,44.999271


## Abkürzungen ==========
* abspalten
* belöschen
* korrigieren
* variieren
* pflegen

In [93]:
geschütze_abkürzungen = ['H2O','DEU','deu','RAF','UNO','CDU','SPD','FDP',
                         'SED','UTC','ORF','RKI',
                         'Mo','Di','Mi','Do','Fr','Sa','So',  'MO','DI','MI','DO','FR','SA','SO',  'Mo.','Di.','Mi.','Do.','Fr.','Sa.','So.', 'NS','US','VR','IR','UV','9/11','OP','KI','AI','IT','IS',
                         'Dax','FDJ','MDR','USB','2D','3D','OEM','VW',
                         'u.','ca.','ev.','PKK','USK','dpi','SUV', 'Fa.','od.','u.','CO2','SO2','VR China','Meck-Pomm','Hbf',
]

In [94]:
bsp_abkürzungen_kurz = ['altisländ.', 'EU','Krh', 'Krh.', 'Krhs', 'Krhs.', 'Krkhs', 'Krkhs.']
bsp_abkürzungen_lang = ['altisländisch', 'Europäische Union','Krankenhaus']

In [95]:
# wiktionary_lemma kann sowohl die Langform als auch die Kurzform als eigenständiges lemma enthalten.
# Eine Kurzform ist dabei mit member 'Abkürzung' gekennzeichnet.
# TODO: Beide müssen nicht unbedingt den gleichen tag haben, siehe 'EU' vs. 'Europäische Union': NN / NE
# TODO: Nicht jede Kurzform hat einen solchen Datensatz.

search_str(wiktionary_lemma, bsp_abkürzungen_kurz + bsp_abkürzungen_lang)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score
25535,EU,2594,0,EU,eu,True,1,NN,NN,rvszqXEgkg,NOM,N,Abkürzung noPl,,95.445206
29756,Europäische_Union,12532,0,Europäische Union,europäische union,True,2,NE,NE,8lJ8G2jJkm,NAM,N,adjNN Wortverbindung adjNN,,18.615679
55175,Krankenhaus,732,0,Krankenhaus,krankenhaus,True,1,NN,NN,ieOk6mCDNg,NOM,N,,,4093.747803
55532,Krh.,885631,0,Krh.,krh.,True,1,TODO,TODO,UN6AKQvjYA,X_todo,X,Abkürzung,,4.753098
55533,Krhs.,885632,0,Krhs.,krhs.,True,1,TODO,TODO,MzaRIaBB1G,X_todo,X,Abkürzung,,5.103037
55855,Krkhs.,885633,0,Krkhs.,krkhs.,True,1,TODO,TODO,dLOgFLtdWF,X_todo,X,Abkürzung,,4.774970
114147,altisländ.,21498,0,altisländ.,altisländ.,False,1,TODO,TODO,XtfE3VzDG1,X_todo,X,Abkürzung,,28.301094
114148,altisländisch,609806,0,altisländisch,altisländisch,False,1,ADJA,ADJA,Mei2C2z07Z,ADJ,A,noSteig,,46.926975


In [96]:
# wiktionary_merkmal enthält für Kurzformen ein Merkmal 'def', das die Langform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_kurz)
mask2 = wiktionary_merkmal.merkmal == 'def'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
53844,EU,def,Europäische_Union,Europäische Union,,NN,1,,15.607533,False
197919,altisländ.,def,altisländisch,altisländisch,,TODO,1,,13.850547,False
1451747,Krh.,def,Krankenhaus,Krankenhaus,,TODO,1,,2.076549,False
1451748,Krhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,2.251519,False
1451749,Krkhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,2.087485,False


In [97]:
# wiktionary_merkmal enthält für Langformen Merkmale 'abk', das eine Kurzform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_lang)
mask2 = wiktionary_merkmal.merkmal == 'abk'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh,,NN,1,,227.105713,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,,NN,1,,227.105713,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,,NN,1,,227.105713,True
1106271,altisländisch,abk,altisländisch~non,non,,ADJA,1,,7.340219,True
1106272,altisländisch,abk,altnordisch,altnordisch,,ADJA,1,,7.340219,True


### wiktionary_merkmal_abk abspalten

In [98]:
#suche_debug = ['au','ei','hm','qm','ui','mg','km','cm','mm','dm','ft','nm','pt','m²','m³','kg','lb','db']

In [99]:
# abk-Merkmale abspalten
mask = (wiktionary_merkmal.merkmal == 'abk')
wiktionary_merkmal, wiktionary_merkmal_abk = pak.move_rows( wiktionary_merkmal, mask, msg=None )
wiktionary_merkmal_abk = wiktionary_merkmal_abk.sort_values(['score','data'], ascending=[False,True])

Move 5107 rows from 4855373


In [100]:
#TODO: Direkt durch die Kurzformen in wiktionary_lemma ergänzen?

In [101]:
# suche_debug wiktionary_merkmal_abk
print(bsp_abkürzungen_lang)
search_str(wiktionary_merkmal_abk, bsp_abkürzungen_lang)

['altisländisch', 'Europäische Union', 'Krankenhaus']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh,,NN,1,,227.105713,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,,NN,1,,227.105713,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,,NN,1,,227.105713,True
1106272,altisländisch,abk,altnordisch,altnordisch,,ADJA,1,,7.340219,True
1106271,altisländisch,abk,altisländisch~non,non,,ADJA,1,,7.340219,True


In [102]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex


### wiktionary_merkmal_abk belöschen

In [103]:
# belöschen: Manuelle Löschungen aus data oder lemma_id
löschmich =  ['Objekt','ECO-Codes','Bibo','Kilom','Land','Basche','DV-Vertrag','Igel','Dolle','Ampere','Baum',
              'Rat','Reibe','Heinrich_1','Diastema mediale','Divisionär','Soda-Zitron','Antillen-Gulden',
              "'H'B","'H'GW","'H'H","'H'L","'H'ST","'H'RO","'H'WI",'gel','gel.','Reg.','Imp.','Reg','Imp','AUR',
             ]

mask = wiktionary_merkmal_abk.data.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 48)

msg1 = 'löschmich_data'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg1)

mask = wiktionary_merkmal_abk.lemma_id.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 65)

msg2 = 'löschmich_id'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg2)

# anschauen
spalten = ['data', 'meta', 'lemma_id',  'lemma_tag',   'merkmal', 'msg']
mask = wiktionary_merkmal_trash.msg.isin([msg1, msg2])
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  48 rows
Move 48 rows from 5107
check_mask:  65 rows
Move 65 rows from 5059
113 rows out of 125


,data,meta,lemma_id,lemma_tag,merkmal,msg
95044,Bibo,umgangssprachliche,Bibliothek,NN,abk,löschmich_data
32407,Kilom,,Kilometer,NN,abk,löschmich_data
116869,Reg,,Regierung,NN,abk,löschmich_data
434639,Reg,,Register,NN,abk,löschmich_data
102414,Reg,,Region,NN,abk,löschmich_data
...,...,...,...,...,...,...
206228,Heyns,,Heinrich_1,NE,abk,löschmich_id
206229,Heynsen,,Heinrich_1,NE,abk,löschmich_id
206230,Hinner,,Heinrich_1,NE,abk,löschmich_id
350072,ANG,,Antillen-Gulden,NN,abk,löschmich_id


In [104]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 6                    # zu lang
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask5 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask6 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask7 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask(wiktionary_merkmal_abk,mask, 338)

msg = 'zulang_single'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  338 rows
Move 338 rows from 4994
338 rows out of 463


,data,meta,lemma_id,lemma_tag,merkmal,msg
46514,Energie,,Arbeit,NN,abk,zulang_single
46513,englisch,,Arbeit,NN,abk,zulang_single
103011,Rechenzeichen,,positiv,ADJA,abk,zulang_single
103010,Vorzeichen,,positiv,ADJA,abk,zulang_single
76349,Flächeneinheit,,Morgen,NN,abk,zulang_single
...,...,...,...,...,...,...
162351,Kraftfahrzeugkennzeichen,,Halle_1,NE,abk,zulang_single
768078,Macchiato,,Latte_macchiato,NN,abk,zulang_single
295492,Aquilin,,Aquilinus,NE,abk,zulang_single
281326,NatGerm,,Nataler_Deutsch,NN,abk,zulang_single


In [105]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 15                   # zu lang
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 10)

msg = 'zulang'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  10 rows
Move 10 rows from 4656
10 rows out of 473


,data,meta,lemma_id,lemma_tag,merkmal,msg
97032,Ohmscher Widerstand,Größenzeichen,Widerstand,NN,abk,zulang
67844,Doctor theologiae,,Doktor,NN,abk,zulang
1100563,Muster-ADV-Vertrag,,Muster-Auftragsdatenverarbeitungsvertrag,NN,abk,zulang
161836,Residual Current Device,englisch,Fehlerstromschutzschalter,NN,abk,zulang
201556,Sacer Ordo Cisterciensis,,Zisterzienser,NN,abk,zulang
199212,Ordo Cartusiensis,,Kartäuser,NN,abk,zulang
1152351,Kfz-Haftpflichtversicherung,,Kraftfahrzeug-Haftpflichtversicherung,NN,abk,zulang
245749,Niederländischen Antillen,Internet,Aruba,NE,abk,zulang
331594,chemische Formel,,Natriumsulfat,NN,abk,zulang
903360,chemische Formel,,Natriumnitrit,NN,abk,zulang


In [106]:
# belöschen: Namen
mask1 =  wiktionary_merkmal_abk.lemma_tag == 'NE'                  # ist Name
mask2 = ~wiktionary_merkmal_abk.data.str.isupper()                 # nicht nur groß
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal_abk,mask, 330)

msg = 'Namen'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  329 rows
Move 329 rows from 4646
329 rows out of 802


,data,meta,lemma_id,lemma_tag,merkmal,msg
4903,Dtl,,Deutschland,NE,abk,Namen
4904,Dtld,,Deutschland,NE,abk,Namen
78528,Bbg,,Brandenburg,NE,abk,Namen
73358,Bln,,Berlin,NE,abk,Namen
73357,Kfz-Kennzeichen,,Berlin,NE,abk,Namen
...,...,...,...,...,...,...
272234,Kfz-Kennzeichen,,Hann._Münden,NE,abk,Namen
1669219,3920,Postleitzahl,Groß_Gerungs,NE,abk,Namen
248823,bbc,ISO,Batak_Toba,NE,abk,Namen
249498,iba,ISO,Iban_2,NE,abk,Namen


In [107]:
# belöschen: Nur Ziffern
mask1 = wiktionary_merkmal_abk.data.str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 20)

msg = 'nur_Ziffern'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  20 rows
Move 20 rows from 4317
20 rows out of 822


,data,meta,lemma_id,lemma_tag,merkmal,msg
544347,102,,hundertundzweite,ADJA CARD,abk,nur_Ziffern
542226,101,,einhunderterste,ADJA CARD,abk,nur_Ziffern
609448,9000,arabische,neuntausend,CARD,abk,nur_Ziffern
541691,100,,einhundertste,ADJA CARD,abk,nur_Ziffern
544350,200,,zweihundertste,ADJA CARD,abk,nur_Ziffern
635132,102,,hundertzweite,ADJA CARD,abk,nur_Ziffern
542239,101,,hunderterste,ADJA CARD,abk,nur_Ziffern
633129,300,,dreihundertste,ADJA CARD,abk,nur_Ziffern
544352,500,,fünfhundertste,ADJA CARD,abk,nur_Ziffern
544355,1000,,tausendste,ADJA CARD,abk,nur_Ziffern


In [108]:
# belöschen: Endet mit Ziffer
mask1 =  wiktionary_merkmal_abk.data.str[-1].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(('²','³','₂')) 
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5 
check_mask(wiktionary_merkmal_abk,mask, 162)

msg = 'Endet_mit_Ziffer'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  162 rows
Move 162 rows from 4297
162 rows out of 984


,data,meta,lemma_id,lemma_tag,merkmal,msg
16159,E 949,E-Nummer,Wasserstoff,NN,abk,Endet_mit_Ziffer
20801,E 175,,Gold,NN,abk,Endet_mit_Ziffer
126543,E 174,,Silber,NN,abk,Endet_mit_Ziffer
16169,E 939,,Helium,NN,abk,Endet_mit_Ziffer
125462,E 948,,Sauerstoff,NN,abk,Endet_mit_Ziffer
...,...,...,...,...,...,...
311747,0-0,,kleine_Rochade,NN,abk,Endet_mit_Ziffer
305638,0-0,,kurze_Rochade,TODO,abk,Endet_mit_Ziffer
358976,0-0,,klein_rochieren,TODO,abk,Endet_mit_Ziffer
305798,0-0-0,,lange_Rochade,TODO,abk,Endet_mit_Ziffer


In [109]:
# belöschen: Bibel
mask1 =  wiktionary_merkmal_abk.lemma_id.str[0].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 12)

msg = 'Bibel'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  12 rows
Move 12 rows from 4135
12 rows out of 996


,data,meta,lemma_id,lemma_tag,merkmal,msg
1773844,"2,5G",,"2,5G-Regel",NN,abk,Bibel
663739,2 Kor,,2._Korintherbrief,NN,abk,Bibel
663720,1 Kor,,1._Korintherbrief,NN,abk,Bibel
1217629,1 Thess,,1._Thessalonicherbrief,NN,abk,Bibel
1217633,2 Thess,,2._Thessalonicherbrief,NN,abk,Bibel
1217648,1 Tim,,1._Timotheusbrief,NN,abk,Bibel
1217654,2 Tim,,2._Timotheusbrief,NN,abk,Bibel
1219019,2 Petr,,2._Petrusbrief,NN,abk,Bibel
1219017,1 Petr,,1._Petrusbrief,NN,abk,Bibel
1219084,1 Joh,,1._Johannesbrief,NN,abk,Bibel


In [110]:
# belöschen: meta
löschmich_meta = ['Postleitzahl','Elementsymbol','seltener','selten','Ordenskürzel','Österreich','Makrosprache',
                  'Kirmanjki','Schweiz','ISO','Kfz','Internet','ISO-3166','Kfz-Kennzeichen','fachsprachlich','Dreibuchstabencode',
                  'international','Deutschland','ISO-4217-Code','ISO-639-3','chemisches','Zentral-Quiche','Komi',
                  'Kraftfahrzeugkennzeichen','englisch','veraltet','Dimli','Shimaore','bisweilen',
                  'Katalogbezeichnungen','ISO-639-1','inoffiziell','olympisch','chemische','Zusammensetzungen',
                  'Netzjargon','ISO-4217','bundesdeutsch','Währungscodes','UN LOCODE','römische','schweizerisch',
                  'Währungscode','Römische','bedeutungsgleiche','Summenformel',
                  'Rumänien','Lüftungstechnik','Lebensmittelfarbstoff','internationaler','volkstümlich',
                  'Maya-Sprachen','ISO-3166-1','ISO-3166-Code','üblich','wissenschaftlich','ehem',
                  'Polizeijargon','IOC-Ländercode','Moldawien','taktisches','postalisch','Abkürzung','Römisches',
                  'fachsprachlich schriftsprachlich','Schifffahrt','Codierung','SIL-Code']
mask1 =  wiktionary_merkmal_abk.meta.isin(löschmich_meta)
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal_abk,mask, 1035)

msg = 'meta'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = wiktionary_merkmal_trash[mask].sort_values(['meta','score'],ascending=[True,False])
grid(a)

check_mask:  1035 rows
Move 1035 rows from 4123
1035 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex,msg
749186,Sperrzone,abk,Sperrzone~SpZo,SpZo,,NN,1,Abkürzung,10.238153,True,meta
628703,Asthenie,abk,ICD,ICD,,NN,1,Codierung,10.136087,True,meta
1520458,Trinkwasserverordnung,abk,Trinkwasserverordnung~TWV,TWV,,NN,1,Deutschland,17.375456,True,meta
103075,Rangierbahnhof,abk,Rbf,Rbf,,NN,,Deutschland,17.273390,True,meta
1133574,Gesundheitsministerium,abk,Gesundheitsministerium~BMG,BMG,,NN,1,Deutschland,10.883353,True,meta
...,...,...,...,...,...,...,...,...,...,...,...
655098,Kubikzentimeter,abk,cbcm,cbcm,,NN,1,veraltet,5.211421,True,meta
655097,Kubikzentimeter,abk,ccm,ccm,,NN,1,veraltet,5.211421,True,meta
175355,Bauernhof,abk,Hof,Hof,,NN,1,volkstümlich,20.539490,True,meta
335119,Wisent,abk,Wisent~B. bonasus,B. bonasus,,NN,1,wissenschaftlich,12.829890,True,meta


In [111]:
# Übriggebliebene nach meta
a = pak.analyse_freqs( wiktionary_merkmal_abk, ['meta','lemma_tag','data'] )
grid(a)

16 rows


,meta,meta_count,meta_percent,lemma_tag,lemma_tag_count,data,data_count
0,,2953,95.6,"[NN, ADJA, NE, ADV, TODO, VVFIN, APPR, , KOMBI, CARD, KON, NN NE, PDAT, ZFIX, PTKANT, ADV APPR, PIAT, KOUS, APPO APPR, ITJ]","[1997, 609, 123, 98, 36, 23, 18, 10, 8, 5, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1]","[Str, inf, Pfl, Konv, ind, Ins, Gen, imp, Chemie, erk, Ztr, Adj, Konj, Chr, ged, get, Pat, NaCl, Ver, Fut]","[16, 9, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4]"
1,manuell,112,3.6,"[NN, KOMBI, ADJA, ADV, KON, $(, NN NE, VVFIN, KOUS, CARD, NE, ADV ADJD, APPR]","[68, 17, 7, 4, 3, 3, 2, 2, 2, 1, 1, 1, 1]","[Bd., Feb., Jan., FR, Str., Nr., std., SO, geschl., OP, Jun., DI, MI, DO, km, Abschn., Hg., Apr., SA, Abb.]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,umgangssprachlich,4,0.1,"[NN, PIAT]","[3, 1]","[was, Perso, Mini, Kelle]","[1, 1, 1, 1]"
3,kurz,3,0.1,NN,3,"[Mayo, Emu, Snooker-WM]","[1, 1, 1]"
4,international,2,0.1,NE,2,"[DEU, UNO]","[1, 1]"
5,physikalisch,2,0.1,NN,2,"[atm, atü]","[1, 1]"
6,chemische,2,0.1,NN,2,"[CO2, SO2]","[1, 1]"
7,Einheitenzeichen,2,0.1,NN,2,"[min, var]","[1, 1]"
8,Summenformel,1,0.0,NN,1,H2O,1
9,ISO,1,0.0,NN,1,deu,1


In [112]:
# meta ist jetzt sinnlos, bis auf 'manuell'
mask =  wiktionary_merkmal_abk.meta != 'manuell'
wiktionary_merkmal_abk.loc[mask,'meta'] = ''

In [113]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex


### Analysieren

In [114]:
# Gelöschte überprüfen, aus suche_debug?
print(suche_debug)
search_str(wiktionary_merkmal_trash, suche_debug)

['calvinistisch', 'kalvinistisch']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex,msg


In [115]:
# Gelöschte überprüfen, aus lexeme_manuell?
mask1 = wiktionary_merkmal_trash.lemma_id.str.endswith('_M') 
mask2 = ~wiktionary_merkmal_trash.msg.isin(['dup'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal_trash, mask, 0)
#wiktionary_merkmal_trash[mask]

check_mask:  0 rows


In [116]:
# Gelöschte überprüfen, aus geschütze_abkürzungen?
mask = wiktionary_merkmal_trash.data.isin(geschütze_abkürzungen)
check_mask(wiktionary_merkmal_trash, mask, 0)

check_mask:  0 rows


In [117]:
# Gelöschte anschauen
a = pak.analyse_freqs( wiktionary_merkmal_trash, ['msg','lemma_tag','data'] )
grid(a)

11 rows


,msg,msg_count,msg_percent,lemma_tag,lemma_tag_count,data,data_count
0,meta,1035,51.0,"[NN, NE, ADJA, CARD, ADV, VVFIN, ITJ]","[769, 247, 8, 5, 4, 1, 1]","[chi, zho, kom, nya, ber, ful, bel, kpv, srn, diq, nbl, nde, smi, roa, NGC, tut, fur, ndo, grc, spa]","[11, 11, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2]"
1,zulang_single,338,16.6,"[NN, ADJA, NE, VVFIN, TODO, PIAT, CARD, APPR, ADV]","[192, 105, 30, 4, 3, 1, 1, 1, 1]","[Summenformel, Kraftfahrzeugkennzeichen, Linguistik, Hirnnerv, alphanum, Wasserl, Ostubaf, Halbstrukturformel, Gruppenformel, Schussw, Fernschr, Morphonem, unverbr, Energie, englisch, Rechenzeichen, Vorzeichen, Flächeneinheit, Großbritannien, Landsch]","[14, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]"
2,Namen,329,16.2,NE,329,"[Kfz-Kennzeichen, Balder, Benz, Ofr, Bert, Chris, UMa, Baldo, UMi, Dtl, Dtld, Bbg, Bln, Lzg, Johs, Österr, Vbg, Afr, Frankr, Frk]","[20, 6, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Endet_mit_Ziffer,162,8.0,"[NN, NE, TODO, VVFIN]","[103, 53, 4, 2]","[ICD-10, 0-0, 0-0-0, ISO 3166-2, OR-1, A-2384, A-2325, A-2351, E 507, ISO 639-3, E 949, E 175, E 174, E 939, E 948, A-2340, G40, E 173, E 501, A-5280]","[19, 6, 4, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,löschmich_id,65,3.2,"[NE, NN]","[61, 4]","[Div, Sozi, Hein, Heincke, Heindel, Heindl, Heine, Heinecke, Heineke, Heinel, Heinelt, Heinemann, Heinen, Heiner, Heinermann, Heinert, Heines, Heinicke, Heinig, Heink]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,löschmich_data,48,2.4,"[NN, ADJA, NE, VVFIN]","[34, 9, 3, 2]","[gel, Reg, Imp, Rat, AUR, Reibe, Bibo, Kilom, Land, Basche, ECO-Codes, Objekt, DV-Vertrag, Igel, Dolle, Ampere, Baum]","[11, 9, 9, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,nur_Ziffern,20,1.0,"[ADJA CARD, CARD, ADV]","[12, 6, 2]","[100, 102, 101, 200, 300, 1000, 9000, 500, 700, 900, 10000, 400, 100000]","[3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]"
7,Bibel,12,0.6,NN,12,"[2,5G, 2 Kor, 1 Kor, 1 Thess, 2 Thess, 1 Tim, 2 Tim, 2 Petr, 1 Petr, 1 Joh, 2 Joh, 3 Joh]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,zulang,10,0.5,"[NN, NE]","[9, 1]","[chemische Formel, Ohmscher Widerstand, Doctor theologiae, Muster-ADV-Vertrag, Residual Current Device, Sacer Ordo Cisterciensis, Ordo Cartusiensis, Kfz-Haftpflichtversicherung, Niederländischen Antillen]","[2, 1, 1, 1, 1, 1, 1, 1, 1]"
9,Leerstring data Müll,6,0.3,"[NE, NN]","[4, 2]",,6


In [118]:
# Gelöschte mit dem höchsten score
mask = ~wiktionary_merkmal_trash.msg.isin(['dup'])
grid(wiktionary_merkmal_trash[mask].sort_values('score',ascending=False))

2031 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex,msg
592833,Blindgänger,abk,Blindgänger~ERW,ERW,,NN,1,international,118.693802,True,meta
592834,Blindgänger,abk,Blindgänger~MEC,MEC,,NN,1,international,118.693802,True,meta
592835,Blindgänger,abk,Blindgänger~UXB,UXB,,NN,1,international,118.693802,True,meta
592836,Blindgänger,abk,Blindgänger~UXO,UXO,,NN,1,international,118.693802,True,meta
46514,Arbeit,abk,Energie,Energie,,NN,7,,100.755768,True,zulang_single
...,...,...,...,...,...,...,...,...,...,...,...
230543,Bit_1,abk,Bit_1~bgk,bgk,,NE,1,ISO,1.250304,True,Namen
615890,West_Berawan,abk,West_Berawan~zbw,zbw,,NN,1,ISO,1.216282,True,meta
460570,geliefert,abk,gel,gel,,ADJA,,,0.628571,True,löschmich_data
430967,gelandet,abk,gel,gel,,ADJA,,,0.595918,True,löschmich_data


In [119]:
# Verbliebene nach tag
pak.analyse_freqs(wiktionary_merkmal_abk,['lemma_tag','meta','data'])

,lemma_tag,lemma_tag_count,lemma_tag_percent,meta,meta_count,data,data_count
0,NN,2085,67.5,"[, manuell]","[2017, 68]","[Str, Pfl, Konv, Ins, Gen, Ztr, Adj, Konj, Pat, NaCl, Ver, Fut, Chr, Erl, StB, Demogr, Geb, Hdl, Anl, Pos]","[16, 7, 7, 6, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3]"
1,ADJA,616,19.9,"[, manuell]","[609, 7]","[ind, ged, get, unv, afr, imp, inf, geh, med, vsl, gebr, ungen, soz, reg, frdl, akt, fries, pers, hurr, isl]","[7, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]"
2,NE,126,4.1,"[, manuell]","[125, 1]","[LUX, RUS, BAD, MKD, ABG, GKW, HAL, DEU, BRB, CHN, AUS, CHE, L.E., MUC, ISO, AFG, AUT, FRA, GBR, GRC]","[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,ADV,102,3.3,"[, manuell]","[98, 4]","[bes, pass, insb, insbes, ebd, östl, dto, sod, bisl, eig, eigtl, teilw, bspw, zur, asb, desgl, dzt, sof, min, mind]","[3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,TODO,36,1.2,,36,"[Chemie, ad lib, s. A., s.A., sel. A., sel.A., gültig, S. M., m. E., i. E., C.c., S. Em., S. H., m. m., z. H., z. Hd., HDTV, a. t., a. l., ad l]","[4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,VVFIN,25,0.8,"[, manuell]","[23, 2]","[erk, inf, imp, gez, abgez, abn, rad, zit, anschl, entf, herst, betr, vgl, vgl., beh, vergl, geb., abr, dest, rat]","[4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,KOMBI,25,0.8,"[manuell, ]","[17, 8]","[i. Allg., Anm. d. Red., i. A., i.A., k.A., o. B., o. dgl., o.ä., s. a., s. o., s. u., s.a., s.o., s.u., u. a., u. dgl., u.U., u.a., u.s.w., u.ä.]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7,APPR,19,0.6,"[, manuell]","[18, 1]","[entg, innerh, anl, vs., bez, bezgl, bzgl, mod, betr, hins, inf, gem, zzgl, abzgl, inkl, südl, westl, östl, exkl]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,,10,0.3,,10,"[dies, nen, inf, jdm, jmdm, jdn, jmdn, jds, jmds]","[2, 1, 1, 1, 1, 1, 1, 1, 1]"
9,KON,7,0.2,"[, manuell]","[4, 3]","[&, bezw, bzw, bzw., d.h., resp, rsp]","[1, 1, 1, 1, 1, 1, 1]"


In [120]:
# suche_debug wiktionary_merkmal_abk
print(suche_debug)
search_str(wiktionary_merkmal_abk, suche_debug)

['calvinistisch', 'kalvinistisch']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex


In [121]:
# suche_debug wiktionary_merkmal
#mask = wiktionary_merkmal.merkmal.isin(['gm'])
#a = search_str(wiktionary_merkmal[mask], suche_debug)
#grid(a.sort_values('lemma_id'))

### wiktionary_merkmal_abk korrigieren

In [122]:
# Einzelne Worte löschen
r = ['(grad)',]
wiktionary_merkmal_abk['data'] = pak.remove_str(wiktionary_merkmal_abk.data, r)

In [123]:
# korrigieren: Punkt innen ohne Punkt hinten
# Das ist eigentlich schon weiter oben passiert
endet_nicht_mit = ( '.', 'Tag', 'Ort',)
mask1 =  wiktionary_merkmal_abk.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(endet_nicht_mit)
mask3 = (wiktionary_merkmal_abk.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask4 =  wiktionary_merkmal_abk.data.str.len() > 2
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal_abk, mask, 0,136 )
wiktionary_merkmal_abk.loc[mask,'data'] += '.'
grid(wiktionary_merkmal_abk,mask)

check_mask:  2 rows
2 rows out of 3088


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
186271,Steuerberater,abk,Steuerberater~St.-Ber,St.-Ber.,,NN,1,,10.803159,True
313841,Steuernummer,abk,Steuernummer~St.-Nr,St.-Nr.,,NN,1,,4.741191,True


In [124]:
# korrigieren: Punkt hinten ergänzen
ausnahmen =  [ 'Admin','Schirm','Kappe','Symbol','Potenz','Schiri','Info','Bus','Kiga','Hof','Teen','Teenie','Teeny','Nitro','Steno','Perso','Navi',]
ausnahmen += [ 'Mayo','Hasch','Britta','Tusse','Tussi','Jäger','Reha','Latte','Pkw','Escape','Spasti','Proll','Prolo','Traufe','Qbit','Qubit',]
ausnahmen += [ 'Rose','Kino','Alster','Hafen','Psycho','Dame','Kelle','Kopie','FeWo','Chemie','Knobi','Makro','Majo','Tief']
mask1 = ~wiktionary_merkmal_abk.data.isin(ausnahmen)
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')                                                       # Kein Bindestrich
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')                                                       # kein Space
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)                                          # kein Punkt
mask5 = ~wiktionary_merkmal_abk.meta.isin(['ISO','kurz','umgangssprachlich','Netzjargon'])
mask6 = (wiktionary_merkmal_abk.data.str.len() > 2)   |   wiktionary_merkmal_abk.data.isin(['Dr','No'])
mask7 =  wiktionary_merkmal_abk.data.str.contains('^[A-Za-zöüäÖÄÜ]')                                         # startswith Buchstabe
mask8 =  wiktionary_merkmal_abk.data.str.count('[A-ZÜÖÄ]')  <=    1                                          # Nicht mehr als 1 Großbuchstabe
mask9 = ~wiktionary_merkmal_abk.data.str.contains('[^A-Za-zöüäÖÄÜß]')                                        # irgendwelche Sonderzeichen

mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7  &  mask8  &  mask9
check_mask( wiktionary_merkmal_abk, mask, 1815 )

wiktionary_merkmal_abk.loc[mask,'data'] += '.'  
grid(wiktionary_merkmal_abk,mask)

check_mask:  1815 rows
1815 rows out of 3088


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh.,,NN,1,,227.105713,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs.,,NN,1,,227.105713,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs.,,NN,1,,227.105713,True
314045,Synagoge,abk,Synagoge~Syn,Syn.,,NN,,,214.588089,True
138965,Garten,abk,Garten~Gart,Gart.,,NN,1,,166.136093,True
...,...,...,...,...,...,...,...,...,...,...
576561,gedämpft,abk,gedämpft~ged,ged.,,ADJA,,,0.583673,True
460882,gepaart,abk,gepaart~gep,gep.,,ADJA,,,0.563265,True
457318,infiziert,abk,infiziert~inf,inf.,,ADJA,,,0.536735,True
479907,getilgt,abk,get_M,get.,,ADJA,,,0.495918,True


In [125]:
# variieren: Punkt mit Space
#mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask = wiktionary_merkmal_abk.data.str.count('\.') > 1
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'. ',   '-':''}
df['data'] = pak.replace_str(df.data, r).str.strip()
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

162 rows added, now a total of 3250


In [126]:
# variieren: Bindestriche weglassen
mask1 =  wiktionary_merkmal_abk.data.str.contains('-')
mask2 = ~wiktionary_merkmal_abk.data.str.startswith('-')
mask3 = ~wiktionary_merkmal_abk.data.str.endswith('-')
mask = mask1  &  mask2  &  mask3
df = wiktionary_merkmal_abk[mask].copy()
r = {'-':' ', '  ':' '}
df['data'] = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

123 rows added, now a total of 3373


In [127]:
# variieren: Punkte weglassen
mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'',}
df['data'] = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

2237 rows added, now a total of 5610


In [128]:
# variieren: Spaces weglassen
mask = wiktionary_merkmal_abk.data.str.contains(' ')
df = wiktionary_merkmal_abk[mask].copy()
r = {' ':'',}
df['data'] = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

697 rows added, now a total of 6307


In [129]:
# pflegen: Überfüssige Spaces streichen
r = {'  ':' ', '  ':' ',}
wiktionary_merkmal_abk['data'] = pak.replace_str(wiktionary_merkmal_abk.data, r)
wiktionary_merkmal_abk['data'] = wiktionary_merkmal_abk.data.str.strip()

In [130]:
grid(wiktionary_merkmal_abk.sort_values('data'))

6307 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,score,is_lex
680,Prozent,abk,Prozent~%,%,,NN,,manuell,17.820169,True
570,und,abk,und~&,&,,KON,,manuell,19.507898,True
2862,-prozentig,abk,-prozentig~-%ig,-%ig,,ZFIX,1,,4.252734,True
1053,-schaft,abk,-sch,-sch,,ZFIX,1,,13.486027,True
1054,-schaft,abk,-schaft~-schft,-schft,,ZFIX,1,,13.486027,True
...,...,...,...,...,...,...,...,...,...,...
2722,überschüssig,abk,überschüssig~übsch,übsch.,,ADJA,1,,5.131227,True
1846,Microcontroller_M,abk,μC,μC,,NN,,manuell,8.736330,True
2686,Mikrosievert,abk,Mikrosievert~μSv,μSv,,NN,1,,5.295261,True
1359,Mikrometer,abk,μm,μm,,NN,,manuell,11.222357,True


In [131]:
# pflegen: Dups entfernen

wiktionary_merkmal_abk = pak.group_and_agg( wiktionary_merkmal_abk, 
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',  'data',   'data_id',  'score',   ],
              [ 'group',    'group',      'min',  'group',     'max',   'group',  'group',    'max',     ],
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',  'data',   'data_id',  'score',   ],        
             )


587 rows less, now 5720 rows


In [132]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,score


In [133]:
# anschauen
grid(wiktionary_merkmal_abk.sort_values('score',ascending=False))

5720 rows


,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,score
1981,Krankenhaus,NN,1,abk,,Krh,Krankenhaus~Krh,227.105713
1982,Krankenhaus,NN,1,abk,,Krh.,Krankenhaus~Krh,227.105713
1986,Krankenhaus,NN,1,abk,,Krkhs.,Krankenhaus~Krkhs,227.105713
1985,Krankenhaus,NN,1,abk,,Krkhs,Krankenhaus~Krkhs,227.105713
1984,Krankenhaus,NN,1,abk,,Krhs.,Krankenhaus~Krhs,227.105713
...,...,...,...,...,...,...,...,...
4463,infiziert,ADJA,,abk,,inf.,infiziert~inf,0.536735
4324,getilgt,ADJA,,abk,,get,get_M,0.495918
4325,getilgt,ADJA,,abk,,get.,get_M,0.495918
4259,gedeckt_1,ADJA,,abk,,ged,gedeckt_1~ged,0.479592


### wiktionary_lemma pflegen

In [134]:
# suche_debug 
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score
122752,calvinistisch,572869,0,calvinistisch,calvinistisch,False,1,ADJA,ADJA,vxjkmPSKuu,ADJ,A,,,44.999271


In [135]:
# Alle auflösbaren Abkürzungen mit eigenem Eintrag in wiktionary_lemma
mask_K = wiktionary_merkmal_abk.lemma_id != wiktionary_merkmal_abk.data_id
wiktionary_merkmal_abk[mask_K].tail(3)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,score
5717,üblich,ADJA,1,abk,,übl.,üblich~übl,12.111786
5718,üblicherweise,ADV,1,abk,,üblw,üblicherweise~üblw,10.639125
5719,üblicherweise,ADV,1,abk,,üblw.,üblicherweise~üblw,10.639125


In [136]:
# wiktionary_lemma.member nachtragen
# Nicht jede Abkürzung hat einen eigenen Eintrag in wiktionary_lemma.
# Aber wenn, dann ist dieser mit member 'Abkürzung' gekennzeichnet.

mask1 = ~wiktionary_lemma.member.str.contains('Abkürzung')                       # nicht als Abkürzung gekennzeichnet
mask2 = wiktionary_lemma.lemma_id.isin(wiktionary_merkmal_abk[mask_K].data_id)   # ist aber eine auflösbare Abkürzung
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 169 )

wiktionary_lemma.loc[mask,'member'] = (wiktionary_lemma[mask].member + ' Abkürzung').str.strip()
grid( wiktionary_lemma, mask )

check_mask:  171 rows
171 rows out of 171220


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,score
114,Ann,46782,0,Ann,ann,True,1,NE,NE,FK4Wp9uVd6,NAM,N,Person_vorname Abkürzung,,132.075104
501,Franz,22959,0,Franz,franz,True,1,NE,NE,2Yip281uSJ,NAM,N,Person_vorname Abkürzung,,163.248901
705,Israel,22790,0,Israel,israel,True,1,NE,NE,yGpROtNS2W,NAM,N,Geo Geo_land Abkürzung,,435.509064
717,Jan,12342,0,Jan,jan,True,1,NE,NE,EhTikYXQTk,NAM,N,Person_vorname Abkürzung,,256.269501
890,Lea,106729,0,Lea,lea,True,1,NE,NE,rWydM7PvR1,NAM,N,Person_vorname Abkürzung,,202.352295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154127,sers,15554,0,sers,sers,False,1,ITJ,ITJ,fHgdMAWP67,X_itj,X,Partikel Abkürzung,,7.435966
156777,trans,72179,0,trans,trans,False,1,ADJA,ADJA,xPtoAR9ZUo,ADJ,A,noSteig noFlex Abkürzung,,14.843014
164026,vorm,245680,0,vorm,vorm,False,1,DIV,DIV,JWjvQS36U3,X_div,X,Kontraktion Abkürzung,,5.212394
167880,zur,11289,0,zur,zur,False,1,DIV,DIV,MjbitVOpAK,X_div,X,Kontraktion Abkürzung,,8.296233


### translate_abk
* translate_abk1: Nimmt die in wiktionary_lemma gelisteten Abkürzungen und löst sie über ihr def-Merkmal auf
* translate_abk2: Basiert auf wiktionary_merkmal_abk

#### translate_abk1

In [137]:
# translate_abk1: Kurzformen aus wiktionary_lemma
mask_l = wiktionary_lemma.member.str.contains('Abkürzung')
spalten = ['lemma_id','lemma','tag','doc_tag','member','score',]
translate_abk1 = wiktionary_lemma[mask_l][spalten]
translate_abk1 = pak.rename_col(translate_abk1,'lemma',    'kurz')
translate_abk1 = pak.rename_col(translate_abk1,'lemma_id', 'kurz_id')
translate_abk1 = pak.rename_col(translate_abk1,'tag',      'kurz_tag')
#translate_abk1 = pak.rename_col(translate_abk1,'doc_tag',  'kurz_tag2')
translate_abk1 = pak.rename_col(translate_abk1,'member',   'kurz_member')
translate_abk1 = pak.rename_col(translate_abk1,'score',    'kurz_score')
translate_abk1 = pak.reset_index(translate_abk1)
pak.sample(translate_abk1)

KeyError: "['doc_tag'] not in index"

In [ ]:
# Langform verknüpfen
mask_m = (wiktionary_merkmal.merkmal == 'def') # alle Definitionen, nicht nur die der Abkürzungen

# lang_id
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data_id',
                                              col_rename = 'lang_id',
                                              func =       'first',
                                            )
# lang
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data',
                                              col_rename = 'lang',
                                              func =       'first',
                                            )

# lang_tag
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'lang_tag',
                                            )

# lang_tag2
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'doc_tag',
                                              col_rename = 'lang_tag2',
                                            )

# lang_member
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )

# lang_score
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'score',
                                              col_rename = 'lang_score',
                                            )

translate_abk1['todo'] = 'lemma' 
translate_abk1['rank'] = 1.8 
print()
pak.sample(translate_abk1)

#### translate_abk2

In [ ]:
# translate_abk2: aus wiktionary_merkmal_abk
translate_abk2 = wiktionary_merkmal_abk.copy()
translate_abk2 = pak.drop_cols(translate_abk2,['num','merkmal'])
translate_abk2 = pak.rename_col(translate_abk2,'lemma_id',  'lang_id')
translate_abk2 = pak.rename_col(translate_abk2,'lemma_tag', 'lang_tag')
translate_abk2 = pak.rename_col(translate_abk2,'score',     'lang_score')
translate_abk2 = pak.rename_col(translate_abk2,'data',      'kurz')
translate_abk2 = pak.rename_col(translate_abk2,'data_id',   'kurz_id')
translate_abk2 = pak.rename_col(translate_abk2,'meta',      'kurz_member')
translate_abk2 = pak.reset_index(translate_abk2)
translate_abk2 = pak.move_cols(translate_abk2,['kurz_id','kurz'])
pak.sample(translate_abk2)

In [ ]:
# keine eigene kurz_id
mask = translate_abk2.kurz_id == translate_abk2.lang_id
check_mask(translate_abk2,mask, 0) 
translate_abk2.loc[mask,'kurz_id'] = ''

In [ ]:
# kurz_id ist nicht in wiktionary_lemma
mask1 = ~translate_abk2.kurz_id.isin(wiktionary_lemma.lemma_id)
mask2 =  translate_abk2.kurz_id != ''
mask = mask1  &  mask2 
check_mask(translate_abk2,mask,4503) 
translate_abk2.loc[mask,'kurz_id'] = ''

In [ ]:
# War das richtig?
bgc('orange')

In [ ]:
# kurz_tag
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'kurz_tag',
                                              func =       'first',
                                            )

# kurz_tag2
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'doc_tag',
                                              col_rename = 'kurz_tag2',
                                            )

# kurz_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'kurz_member',
                                              cond =       'null'
                                            )

# kurz_score
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'score',
                                              col_rename = 'kurz_score',
                                            )

# lang
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma',
                                              col_rename = 'lang',
                                            )
# lang_tag2
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'doc_tag',
                                              col_rename = 'lang_tag2',
                                            )


# lang_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )


translate_abk2 = pak.move_cols(translate_abk2,['kurz_tag','kurz_tag2','kurz_member','kurz_score'],'kurz')
translate_abk2 = pak.move_cols(translate_abk2,['lang','lang_tag','lang_tag2','lang_member'],'lang_id')
translate_abk2['rank'] = 1.0 
translate_abk2['todo'] = 'merkmal' 
print()
pak.sample(translate_abk2)

#### translate_abk

In [ ]:
# translate_abk erstellen
translate_abk = pd.concat([translate_abk1,translate_abk2], ignore_index=True)
translate_abk = pak.change_datatype(translate_abk)
translate_abk_trash = translate_abk.head(0)
translate_abk_trash['msg'] = None

In [ ]:
# fillna

translate_abk['kurz_id']     = translate_abk.kurz_id.fillna('')
translate_abk['lang_id']     = translate_abk.lang_id.fillna('')

translate_abk['kurz']        = translate_abk.kurz.fillna('')
translate_abk['lang']        = translate_abk.lang.fillna('')

translate_abk['kurz_tag']    = translate_abk.kurz_tag.fillna('').str.replace('TODO','').str.strip()
translate_abk['lang_tag']    = translate_abk.lang_tag.fillna('').str.replace('TODO','').str.strip()

translate_abk['kurz_tag2']    = translate_abk.kurz_tag2.fillna('')
translate_abk['lang_tag2']    = translate_abk.lang_tag2.fillna('')

translate_abk['kurz_member'] = translate_abk.kurz_member.fillna('')
translate_abk['lang_member'] = translate_abk.lang_member.fillna('')

translate_abk['kurz_score']  = translate_abk.kurz_score.fillna(0)
translate_abk['lang_score']  = translate_abk.lang_score.fillna(0)

#### Belöschen und ranken

In [ ]:
# konkurrenz: Wie viele konkurrierende lang_id gibt es?
gruppiert = translate_abk.groupby(['kurz'])
translate_abk['konkurrenz'] = gruppiert.lang_id.transform( 'nunique' ) 

In [ ]:
#grid(translate_abk)

In [ ]:
# ID fehlt
maskA = translate_abk.kurz_id == ''
maskB = translate_abk.lang_id == ''
mask = maskA | maskB
check_mask(translate_abk,mask,6213)

translate_abk.loc[mask,'rank'] *= 0.5
translate_abk[mask]

In [ ]:
# Anfangsbuchstabe ungleich
mask1 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask2 = ~translate_abk.kurz.str[0].str.isdigit() 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk,mask,992)

translate_abk.loc[mask,'rank'] *= 0.8
translate_abk[mask]

In [ ]:
# nichtssagende_abk1
mask1 =  translate_abk.kurz.str.len() == 3
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(translate_abk,mask,20)

msg = 'nichtssagende_abk1'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# nichtssagende_abk2
mask1 =  translate_abk.kurz.str.len() == 4
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 =  translate_abk.kurz.str.endswith('.')
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,34)

msg = 'nichtssagende_abk2'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# Abk_kürzt_nicht
mask1 =  translate_abk.lang.str.len() <= translate_abk.kurz.str.len() 
mask2 =  translate_abk.lang_id != ''
mask3 =  translate_abk.kurz_id != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,161)

msg = 'Abk_kürzt_nicht'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# zu_kurz_end_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.lower().str.endswith(('a.','e.','i.','o.','u.','ä.','ö.','ü.','y.'))
mask3 =  translate_abk.kurz_tag == ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,20)

msg = 'zu_kurz_end_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# zu_kurz_start_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.endswith('.')
mask3 =  translate_abk.kurz.str.lower().str.startswith(('a','e','i','o','u','ä','ö','ü','y'))
mask4 =  translate_abk.kurz_tag == ''
mask5 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask6 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask(translate_abk,mask,18)

msg = 'zu_kurz_start_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# Ranking: Ist die Abkürzung in lexeme_manuell?
mask = translate_abk.kurz_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

mask = translate_abk.lang_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

In [ ]:
# Ranking: Ist die Abkürzung beidseitig verknüpft?
gruppiert = translate_abk.groupby(['kurz_id'])
translate_abk['rank2'] = gruppiert.todo.transform( 'nunique' ) 

mask = translate_abk.rank2 > 1
translate_abk.loc[mask,'rank'] *= 5.0
translate_abk = pak.drop_cols(translate_abk,'rank2')

In [ ]:
# Abkürzung hat eigenen Eintrag
mask = translate_abk.kurz_id != ''
translate_abk.loc[mask,'rank'] *= 2.0

In [ ]:
# Ranking: lang_score
translate_abk['rank'] *= (translate_abk.lang_score + 0.2)

In [ ]:
# Ranking: Kein tag
mask = translate_abk.lang_tag == ''
translate_abk.loc[mask,'rank'] *= 0.2
translate_abk[mask]

In [ ]:
# sicherheitshalber einmal sortieren
translate_abk = translate_abk.sort_values(['kurz','lang_score'],ascending=[True,False]).reset_index(drop=True)

In [ ]:
# Ranking betrachten
grid(translate_abk.sort_values(['konkurrenz','kurz','rank',],ascending=[False,True,False]))

In [ ]:
# rank!
translate_abk = pak.rank(translate_abk, col_score='rank', cols_group='kurz', on_conflict='first')

In [ ]:
# zu häufig und Anfangsbuchstabe ungleich

erlaubt = ['Ingenieur','okay','per Adresse','Professor','Kalorie','insbesondere','insbesonders']

a = pak.analyse_freqs(translate_abk,'lang')
mask_a = a.lang_count >= 5 
mask1 = translate_abk.lang.isin(a[mask_a].lang)
mask2 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask3 = ~translate_abk.kurz.str[0].str.isdigit() 
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5A = translate_abk.kurz_member == 'Abkürzung'  # genau gleich 'Abkürzung', also nichts zusätzliches 
mask5B = translate_abk.kurz_member == ''           # oder ganz leer 
mask6 = ~translate_abk.lang.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  (mask5A | mask5B)  &  mask6  
check_mask(translate_abk,mask,183)

msg = 'zu_häufig'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
a = pak.analyse_freqs(translate_abk,['lang','kurz'])
grid(a)

In [ ]:
# erlaubt_und_verboten

verboten     = ['Bundesministerium','Brandmeister','Braumeister','Rundfunk','Ortsnamengrundwort','Kurzwort','Diminutiv','diminuendo']
doch_erlaubt = []

mask1 =  translate_abk.lang.isin(verboten) 
mask2 = ~translate_abk.lang.isin(doch_erlaubt) 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4 
#grid(translate_abk,mask)
check_mask(translate_abk,mask,66)

msg = 'erlaubt_und_verboten'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# lang_tag_fehlt
mask1 = translate_abk.lang_tag == ''
mask2 = translate_abk.kurz_tag != ''
mask3 = translate_abk.lang_id  != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(translate_abk,mask)
check_mask(translate_abk,mask,6)

msg = 'lang_tag_fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# tag von lang nach kurz übertragen
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_tag != ''
mask = mask1  &  mask2
check_mask(translate_abk,mask,5068)

translate_abk.loc[mask,'kurz_tag'] = translate_abk[mask].lang_tag
grid(translate_abk,mask)

In [ ]:
# Weiterer_Schrott
mask1 =  translate_abk.kurz_tag == ''
mask2 = ~translate_abk.lang_member.isin(['','Wortverbindung','Redewendung'])
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  
#grid(translate_abk,mask)
check_mask(translate_abk,mask,2)

msg = 'Weiterer_Schrott'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# KOMBI
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member.isin(['Wortverbindung','Redewendung'])
mask = mask1  &  mask2 
check_mask(translate_abk,mask,88)

translate_abk.loc[mask,'kurz_tag'] = 'KOMBI'
grid(translate_abk,mask)

In [ ]:
# lang_member ist jetzt ausgeschöpft
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member != ''
mask = mask1  &  mask2 
check_mask(translate_abk,mask,0)

In [ ]:
# Großgeschrieben => NN
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.kurz.str[0].str.isupper()
mask3  = translate_abk.kurz.str[0].str.isalpha() 
mask4  = translate_abk.kurz.str.len() > 2
mask5A = translate_abk.lang_tag2.str.contains('NN|NE', regex=True)
mask5B = translate_abk.kurz.str.len() >= 4 
mask = mask1  &  mask2  &  mask3  &  mask4  &  (mask5A | mask5B) 
check_mask(translate_abk, mask, 945)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

In [ ]:
# Großgeschrieben => NN
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.kurz.str[0].str.isupper()
mask3  = translate_abk.kurz.str[0].str.isalpha() 
mask4  = translate_abk.kurz.str[0] ==  translate_abk.lang.str[0]
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(translate_abk, mask, 4)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

In [ ]:
# Aufzählungen => CARD
# nicht nur für kurz_tag == ''
mask1 = translate_abk.kurz.str[:-1].str.isnumeric()
mask2 = translate_abk.kurz.str.endswith('.')  
mask3 = translate_abk.kurz.str.len() <= 3 
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk, mask, 11)
translate_abk.loc[mask,'kurz_tag'] = 'CARD'
grid(translate_abk,mask)

In [ ]:
# Rest
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.lang_tag2 == ''
mask = mask1  &  mask2  
check_mask(translate_abk, mask, 719)

grid(translate_abk,mask)
msg = 'Rest'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# Mehr gibt es nicht
mask = translate_abk.kurz_tag == ''
check_mask(translate_abk, mask, 0)

#grid(translate_abk,mask)
#msg = 'Rest'
#translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

In [ ]:
# Neue kurz_id vergeben
mask = translate_abk.kurz_id == ''
translate_abk.loc[mask,'kurz_id'] = translate_abk[mask].kurz.str.replace(' ','_') + '_A'
check_mask(translate_abk,mask,2983)
grid(translate_abk,mask)

In [ ]:
# member Abkürzung nachtragen
mask = ~translate_abk.kurz_member.str.contains('Abkürzung')
translate_abk.loc[mask,'kurz_member'] = (translate_abk[mask].kurz_member + ' Abkürzung').str.strip()

#### rückübertragen

In [ ]:
# Alle Kurzformen aus wiktionary_lemma rauslöschen, die nicht in translate_abk erwähnt werden
# und keinen echten tag haben
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.member.str.contains('manuell')
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask4 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask5 =  wiktionary_lemma.tag == 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(wiktionary_lemma,mask)
check_mask(wiktionary_lemma,mask,1136)

msg = 'translate_abk1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( wiktionary_lemma, wiktionary_lemma_trash, mask, msg)

# anschauen
grid( wiktionary_lemma_trash, msg )

In [ ]:
# Einige Datensätze sind in translate_abk gelöscht, haben aber in wiktionary_lemma einen richtigen tag.
# Die bleiben drin.
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask4 =  wiktionary_lemma.tag != 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma,mask,58)

# anschauen
#grid( wiktionary_lemma, mask )

In [ ]:
# kurz_tag => tag_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'kurz_tag',
                                   col_rename = 'tag_new',
                                   func =       'first',
                                  )
wiktionary_lemma = pak.move_cols(wiktionary_lemma,'tag_new','lemma_len')

In [ ]:
# tag_new => tag
mask = (wiktionary_lemma.tag != wiktionary_lemma.tag_new)  
check_mask(wiktionary_lemma,mask,3014)
wiktionary_lemma.loc[mask,'tag'] = wiktionary_lemma[mask].tag_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'tag_new')

In [ ]:
# pflege_tag_felder für die veränderten tags
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1

In [ ]:
# lang_id => data_id_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'lang_id',
                                   col_rename = 'data_id_new',
                                   func =       'first',
                                  )

In [ ]:
# data_id_new => data_id
mask = (wiktionary_lemma.data_id != wiktionary_lemma.data_id_new)  
check_mask(wiktionary_lemma,mask, 2397)
wiktionary_lemma.loc[mask,'data_id'] = wiktionary_lemma[mask].data_id_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'data_id_new')

In [ ]:
# Neue Datensätze für wiktionary_lemma
mask1 = translate_abk.kurz_id.str.endswith('_A')
mask2 = ~translate_abk.lang_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  
check_mask(translate_abk,mask,2934)

df = translate_abk[mask]

df = pak.rename_col(df, 'kurz_id',     'lemma_id')
df = pak.rename_col(df, 'kurz',        'lemma')
df = pak.rename_col(df, 'kurz_tag',    'tag')
df = pak.rename_col(df, 'kurz_member', 'member')
df = pak.rename_col(df, 'kurz_score',  'score')
df = pak.rename_col(df, 'lang_id',     'data_id')
df['page_id']   = -1
df['page_part'] = -1
df['doc_len']   = ''
df['doc_lemma'] = ''
df['doc_tag']   = ''
df = pak.move_cols(df,['page_id','page_part'],'lemma_id')
df = pak.drop_cols(df,['konkurrenz','rank','todo','lang','lang_tag','lang_tag2','lang_member','lang_score','kurz_tag2',])

df = ergänze_felder(df)
df = pflege_tag_felder(df, translate_tagZ)  
assert set(wiktionary_lemma.columns) ==  set(df.columns) 
grid(df)

In [ ]:
# wiktionary_lemma anfügen!
wiktionary_lemma = pak.add_rows(wiktionary_lemma, df)

In [ ]:
# lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

In [ ]:
# Keine neuen Datensätze für translate_lex
mask1 = translate_abk.kurz_id.str.endswith('_A')
mask2 = translate_abk.lang_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  
check_mask(translate_abk,mask,0)

#### nochmal belöschen

In [ ]:
# lang_id fehlt >> löschen
mask = translate_abk.lang_id == ''
check_mask(translate_abk, mask, 1062)

msg = 'lang_id fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

In [ ]:
# Bedeutungsloses ggf. löschen (dürfte eigentlich nichts mehr dabei sein)
maskA = translate_abk.kurz_id == translate_abk.lang_id
maskB = translate_abk.kurz    == translate_abk.lang
mask = maskA | maskB
check_mask(translate_abk,mask,0)

msg = 'bedeutungslos'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

### wiktionary_merkmal belöschen
* alle abk-Merkmale sind schon gelöscht

In [ ]:
# alle kurzformen
mask_abk =  wiktionary_lemma.member.str.contains('Abkürzung')
kurzformen = wiktionary_lemma[mask_abk].lemma_id

In [ ]:
# Alle def-Merkmale aller Kurzformen löschen
mask1 = wiktionary_merkmal.merkmal == 'def'
mask2 = wiktionary_merkmal.lemma_id.isin(kurzformen)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,4659)

msg = 'def einer Abkürzung'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)

mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash, mask )

### Abkürzungen überprüfen

In [ ]:
search_str(translate_abk,'5G')

In [ ]:
# Alle Kurzformen sind in wiktionary_lemma
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')

mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask,0)

In [ ]:
mask = wiktionary_lemma.lemma_id == 'H2O_A'
wiktionary_lemma[mask]

In [ ]:
raise Stop

## wiktionary_lemma ==========

### pflegen

In [ ]:
# manuelle_kürzel erstellen: Welche Sorten Sonder-IDs gibt es?
# alt: ('_V', '_M', '_N', '_Z', '_O', '_NV')
# ('_V', '_A', '_M', '_N', '_MADJ', '_Z', '_T')

wiktionary_lemma['kürzel'] = wiktionary_lemma.lemma_id.str.split('_').str[-1]

mask = wiktionary_lemma.lemma_id.str.contains('_')
a = pak.analyse_freqs(wiktionary_lemma[mask], ['kürzel','lemma_id'])

mask1 = a.kürzel_count > 30
mask2 = a.kürzel.str.isalpha() 
mask3 = a.kürzel.str.len() < 5 
mask  = mask1  &  mask2  &  mask3
manuelle_kürzel = tuple(['_'+kürzel for kürzel in list(a[mask].kürzel)])  # um manuelle lemma_id erkennen zu können 
print(manuelle_kürzel)
assert len(manuelle_kürzel) == 7
grid(a,mask)

wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'kürzel')

In [ ]:
# score page_part -1 analysieren
mask = wiktionary_lemma.page_part == -1
pak.analyse_freqs(wiktionary_lemma[mask],'member',splits=' ')

In [ ]:
# score page_part -1 pushen
mask = wiktionary_lemma.page_part == -1
check_mask(wiktionary_lemma,mask,7047)

wiktionary_lemma.loc[mask,'score'] += 0.2
wiktionary_lemma.loc[mask,'score'] *= 1.5

### tag

In [ ]:
# sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member','lemma_id'], splits=[' ', None] )

In [ ]:
# NE-Datensätze löschen, die mit hoher Wahrscheinlichkeit was anderes als einen Namen bezeichnen
geschützte_namen = ['Martin','Baum','Junge','Klein','Keller','Mann','Peter','Schwarz','Simon',
                    'Thomas','Weiß','Winter','Sommer','Wolf','Stein','Henry','Frank','Müller',
                    'Meier','Huber','Schneider','Bauer','Conrad','Ackermann','Kramer','Grimm',
                    'Heinrich','Wagner','Pfeifer','Zimmer','Bach','Krämer','Fischer','Arnold',
                    'Lorenz','Fiedler','Weber','Walther','Busch','Schäfer','Michel','Ritter',
                    'Beck',
                   ]
mask1 =  wiktionary_lemma.member.str.contains('Person_nachname')
mask2 =  wiktionary_lemma.tag == 'NE'
mask3 =  wiktionary_lemma.page_part > 0
mask4 =  wiktionary_lemma.score < 2.6
mask5 = ~wiktionary_lemma.lemma.isin(geschützte_namen)
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask( wiktionary_lemma, mask, 343 )

msg = 'NE1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

In [ ]:
# Prüfen: Alle Person_nachname sind NE
mask1 = wiktionary_lemma.member.str.contains('Person_nachname')
mask2 = wiktionary_lemma.tag == 'NE'
mask = mask1  &  ~mask2
check_mask(wiktionary_lemma,mask,0)

In [ ]:
# anschauen
mask = wiktionary_lemma.member.str.contains('Person_nachname')
a = pak.analyse_freqs( wiktionary_lemma[mask], ['member','lemma'] )
grid(a)

In [ ]:
#grid(pak.analyse_cols(wiktionary_lemma))

In [ ]:
# KOMBI: Datensätze umtaggen, die eine Wortverbindung oder eine Redewendung beinhalten
mask1  = wiktionary_lemma.tag == 'TODO'
mask2a = wiktionary_lemma.member.str.contains('Redewendung')
mask2b = wiktionary_lemma.member.str.contains('Wortverbindung')
mask = mask1  &  (mask2a | mask2b)
check_mask( wiktionary_lemma, mask, 407 )

change_tag( wiktionary_lemma, mask, 'KOMBI', manuelle_kürzel )

#wiktionary_lemma.loc[mask,'tag']   = 'KOMBI'
#wiktionary_lemma.loc[mask,'tag_0'] = 'KOMBI'
#wiktionary_lemma.loc[mask,'tagZ']  = 'X_kombi'
#wiktionary_lemma.loc[mask,'tagZZ'] = 'X'
#grid(wiktionary_lemma,mask)

In [ ]:
# Rest sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'lemma_id'], splits=[' ', None] )

In [ ]:
#mask = wiktionary_lemma.member.str.contains('Abkürzung')
#pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'tag'],  )

In [ ]:
# Rest löschen
mask = wiktionary_lemma.tag == 'TODO'
check_mask( wiktionary_lemma, mask, 0, 10 )

msg = 'TODO-Rest'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

In [ ]:
# ADJD >> ADJA
wiktionary_lemma['tag'] = wiktionary_lemma.tag.str.replace('ADJD','ADJA')

In [ ]:
# Einzelkorrekturen        
einzelkorrekturen = { 'sämtlich'       : 'PIAT ADJA',  
                      'besitzergreifen': 'VVFIN',    
                    }
for i,t in einzelkorrekturen.items():
    mask = (wiktionary_lemma.lemma_id == i)
    wiktionary_lemma.loc[mask,'tag'] = t

In [ ]:
# Hilvsverben auf VAFIN
hilfsverben = [ 'haben', 'sein', 'werden', 'tun' ]
mask1 = wiktionary_lemma.tagZZ == 'V'
mask2 = wiktionary_lemma.lemma.isin(hilfsverben)
mask3 = wiktionary_lemma.tag != 'VAFIN'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 1)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VAFIN', manuelle_kürzel )

In [ ]:
# Modalverben auf VMFIN
modalverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 =  wiktionary_lemma.lemma.isin(modalverben)
mask3 =  wiktionary_lemma.tag != 'VMFIN'
mask4 =  wiktionary_lemma.page_part == 0
mask  = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_lemma, mask, 7)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VMFIN', manuelle_kürzel )

In [ ]:
# Obsolet: Alle anderen auf VVFIN 
#TODO vgl. und vergleiche müssten hier verschwinden
sonderverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen', 'haben', 'sein', 'werden', 'tun' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(sonderverben)
mask3 =  wiktionary_lemma.tag != 'VVFIN'
mask  = mask1  &  mask2  &  mask3  
check_mask( wiktionary_lemma, mask, 0)

#change_tag( wiktionary_lemma, mask, 'VVFIN', manuelle_kürzel, manuelle_quellen_schützen=False )
#grid(wiktionary_lemma,mask)

In [ ]:
# Verben analysieren
mask = wiktionary_lemma.tagZZ == 'V'
pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma_id'])

### Affix und Suffix

In [ ]:
# AFIX verschieben
mask1  = wiktionary_lemma.lemma.str.endswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 554)

msg = 'AFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

In [ ]:
# ZFIX verschieben
mask1 = wiktionary_lemma.lemma.str.startswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 389)

msg = 'ZFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

In [ ]:
# Sonstige Fixe
maskA = wiktionary_lemma.lemma.str.startswith('-')
maskB = wiktionary_lemma.lemma.str.endswith('-')
maskC = wiktionary_lemma.tag.str.contains('AFIX')
maskD = wiktionary_lemma.tag.str.contains('ZFIX')
maskE = wiktionary_lemma.member.str.contains('Affix')
maskF = wiktionary_lemma.member.str.contains('Suffix')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = (maskA | maskB | maskC | maskD | maskE | maskF)  &  mask2
check_mask( wiktionary_lemma, mask, 2)

msg = 'DIVFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

### member

In [ ]:
# member Aufzählung bei CARD
mask1 = wiktionary_lemma.lemma.str[:-1].str.isnumeric()
mask2 = wiktionary_lemma.lemma.str.endswith('.')  
mask3 = wiktionary_lemma.lemma.str.len() <= 3 
mask4 = wiktionary_lemma.tag == 'CARD'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma, mask, 11)

wiktionary_lemma.loc[mask,'member'] += ' Aufzählung'
grid(wiktionary_lemma,mask)

In [ ]:
# member noSg und noPl nur bei Nomen
mask1a = wiktionary_lemma.member.str.contains('noPl')
mask1b = wiktionary_lemma.member.str.contains('noSg')
mask2  = wiktionary_lemma.tagZZ != 'N'
mask  = (mask1a | mask1b)  &  mask2
check_mask( wiktionary_lemma, mask, 0, 17 )

wiktionary_lemma.loc[mask,'member'] =  pak.remove_words( wiktionary_lemma[mask].member, ['noPl', 'noSg'] )
#grid(wiktionary_lemma,mask)

### Allgemein

In [ ]:
# Redundante Spalten
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0) # ein paar können wir tolerieren

In [ ]:
# Keine Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
check_mask( wiktionary_lemma, mask, 0)

In [ ]:
# not any_nan
assert not pak.any_nan(wiktionary_lemma)
#pak.analyse_cols(wiktionary_lemma)

In [ ]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

In [ ]:
a = pak.analyse_freqs(wiktionary_lemma,['tag','member'], splits=[None,' '])
grid(a)

## wiktionary_merkmal ==========

### pflegen

In [ ]:
# merkmal_order: Sichert die Reihenfolge der gleichnamigen Merkmale, z.B. mehrere Defs
wiktionary_merkmal = wiktionary_merkmal.reset_index()
wiktionary_merkmal = pak.rank( wiktionary_merkmal, 
                               cols_group = ['lemma_id','merkmal'], 
                               col_score = 'index',
                               find = 'min',
                               col_target = 'merkmal_order')
wiktionary_merkmal = pak.rename_col(wiktionary_merkmal,'index','i')
wiktionary_merkmal = pak.force_index(wiktionary_merkmal,'i')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'merkmal_order','lemma_id')

In [ ]:
wiktionary_merkmal.head(1)

### Schrott löschen 1
* ohne die Felder lemma und lemma_tag

In [ ]:
# zu viele Sonderzeichen
mask1 =  wiktionary_merkmal.data.str.count('[A-ZÖÄÜa-zöäüß0-9]') < wiktionary_merkmal.data.str.count('[^A-ZÖÄÜa-zöäüß0-9]')-1
mask2 =  wiktionary_merkmal.data.str.len() > 2
mask3 = ~wiktionary_merkmal.merkmal.isin(['ipa','rhymes'])
mask4 = ~wiktionary_merkmal.data.isin(['u. s. w.','V. I. P.'])
mask  = mask1  & mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal, mask, 19)
#grid(wiktionary_merkmal, mask)

msg = 'zu viele Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

In [ ]:
# fehlende Klammern
mask1 = wiktionary_merkmal.data.str.count('\(') != wiktionary_merkmal.data.str.count('\)')
mask2 = wiktionary_merkmal.data.str.len() > 1
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0, 10)

msg = 'fehlende_Klammern'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
# Fugenelemente
mask1 =  wiktionary_merkmal.data != '-'
mask2 =  wiktionary_merkmal.data.str.startswith('-')
mask3 =  wiktionary_merkmal.data.str.endswith('-')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

msg = 'Fugenelemente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

### aufsplitten
wiktionary_merkmal wird in drei Teile aufgesplittet:
* wiktionary_merkmal wurzelt in wiktionary_lemma
* wiktionary_lexmerkmal wurzelt in translate_lex.<br>
  translate_lex enthält ja Lexeme (z.B. die Deklinierten Formen). <br>
  wiktionary_lexmerkmal enthält die Merkmale dieser Lexeme.<br>
* translate_gm enthält alle gm-Merkmale, also die Verweise von einer deklinierten Form auf das Lemma.<br>

In [ ]:
# Spalten löschen
wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, ['lemma_lower'])

In [ ]:
# anschauen: verwaistes und verschollenes lemma_id
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_trash.lemma_id)
mask3 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_fix.lemma_id)
mask4 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
grid(wiktionary_merkmal,mask)

In [ ]:
# verwaistes lemma_id
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2

#grid(wiktionary_merkmal,mask)
a = pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','lemma_id'])
grid(a)

In [ ]:
# verwaistes lemma_id löschen
check_mask(wiktionary_merkmal,mask,14420) 

msg = 'verwaist_lemma_id'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
#grid( wiktionary_merkmal_trash, msg )

In [ ]:
# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = pak.analyse_freqs( wiktionary_merkmal_trash[mask], ['merkmal','lemma_tag','data',] )
grid(a)

In [ ]:
# verwaiste data_id (früher wurden die gelöscht) 
mask1 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask3 =  wiktionary_merkmal.data_id != ''
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 51123)
grid(wiktionary_merkmal,mask)

In [ ]:
# translate_gm abspalten
mask = wiktionary_merkmal.merkmal == 'gm'
check_mask(wiktionary_merkmal, mask, 671313)
wiktionary_merkmal, translate_gm = pak.move_rows(wiktionary_merkmal, mask, msg=None)

In [ ]:
# wiktionary_lexmerkmal abspalten
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 =  wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal, mask, 1658317)

wiktionary_merkmal, wiktionary_lexmerkmal = pak.move_rows(wiktionary_merkmal, mask, msg=None)
wiktionary_lexmerkmal = pak.rename_col(wiktionary_lexmerkmal, 'lemma_id','lex_id')

### pflegen

In [ ]:
# suche_debug wiktionary_merkmal
a = search_str(wiktionary_merkmal, suche_debug)
grid(a)

In [ ]:
# lemma nach wiktionary_merkmal übertragen
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')

In [ ]:
# lemma_tag nach wiktionary_merkmal übertragen
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

In [ ]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

### Schrott löschen 2
* diesmal mit den Feldern lemma und lemma_tag

In [ ]:
# beginnt mit Sonderzeichen
erlaubt = ["'nen"]
mask0 = ~wiktionary_merkmal.data.isin(erlaubt)
mask1 =  wiktionary_merkmal.data.str.len() > 1
mask2 = ~wiktionary_merkmal.data.str[0].str.isalpha()  
mask3 = ~wiktionary_merkmal.data.str[0].str.isnumeric()    
mask4 = ~wiktionary_merkmal.lemma_tag.isin([ '$(', '$.', '$,', 'ZFIX NE', 'ZFIX', 'CARD' ])
mask5 =  wiktionary_merkmal.is_lex == True
mask  =  mask0  &  mask1  &  mask2  &  mask3  &  mask4  &  mask5    
check_mask( wiktionary_merkmal, mask, 88)

msg = 'beginnt mit Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg ) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# nicht_erlaubte_sonderzeichen 
mask1 = wiktionary_merkmal.data.str.contains(nicht_erlaubte_sonderzeichen)   
mask2 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2 
check_mask( wiktionary_merkmal, mask, 2)

msg = 'nicht_erlaubte_sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

In [ ]:
# zu viele Wörter 
mask1 =  wiktionary_merkmal.data.str.split().str.len() > 3
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  &  mask3 
check_mask( wiktionary_merkmal, mask, 0)

msg = 'zu viele Wörter'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
# Verbotene Fragmente in data, die nicht im lemma auftreten
verbotene_fragmente = ['Konjugation','Genitiv','Dativ','(',')',':','»','<','!','#',',',';','!','?']
wiktionary_merkmal['error'] = None
for verboten in verbotene_fragmente:
    mask1 =  wiktionary_merkmal.data.str.contains(verboten, regex=False)    
    mask2 = ~wiktionary_merkmal.lemma.str.contains(verboten, regex=False)  
    mask3 =  wiktionary_merkmal.data.str.len() > 1       
    mask4 =  wiktionary_merkmal.is_lex == True   
    mask5 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
    mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5
    wiktionary_merkmal.loc[ mask, 'error'] = 'Verbotenes Fragment ' + verboten

In [ ]:
mask = ~wiktionary_merkmal.error.isnull()  
check_mask( wiktionary_merkmal, mask, 0, 5)
#grid( wiktionary_merkmal, mask)

wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, 'error' )
msg = 'Verbotene Fragmente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
# Überflüssige lemmagleiche Lexeme löschen
mask1  = wiktionary_merkmal.is_lex == True
mask2  = wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3A = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask3B = wiktionary_merkmal.merkmal.isin( ['lex'] )
mask  = mask1  &  mask2  &  (mask3A | mask3B) 
check_mask( wiktionary_merkmal, mask, 205)

msg = 'lemmagleiche Lexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
# Überflüssige lemmagleiche Nichtlexeme löschen
mask1 =  wiktionary_merkmal.is_lex == False
mask2 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3 = ~wiktionary_merkmal.merkmal.isin( ['syllables','Hilfsverb'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 1014)

msg = 'lemmagleiche Nichtlexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
# Benannte Lexeme
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal != 'lex'
mask  = mask1  &  mask2
benannte_lexeme = wiktionary_merkmal[mask]

In [ ]:
# Überflüssige lex: Die gibt es schon unter einem konkreteren Merkmal
# Das sind nicht so viele, weil die alle in translate_lex sind
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.merkmal == 'lex'
mask3 = pak.isin( wiktionary_merkmal, benannte_lexeme, on=['lemma','data'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 21)

del benannte_lexeme
msg = 'Überflüssige lex'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
# TRUNC löschen
mask =  wiktionary_merkmal.lemma_tag == 'TRUNC'
check_mask( wiktionary_merkmal, mask, 0, 4)

msg = 'TRUNC'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
#search_str(wiktionary_lemma, 'http')

In [ ]:
# suche_debug
maskA = wiktionary_merkmal_trash.data.isin(suche_debug)
maskB = wiktionary_merkmal_trash.lemma.isin(suche_debug)
mask = maskA | maskB
grid(wiktionary_merkmal_trash,mask, color='green')

In [ ]:
# suche_debug
#mask = wiktionary_merkmal.data.isin(suche_debug)
mask = wiktionary_merkmal.lemma.isin(suche_debug)
#mask = maskA | maskB
grid(wiktionary_merkmal,mask)

In [ ]:
# Nochmal sicherstellen: Alle Datensätze in wiktionary_merkmal haben einen passenden DS in wiktionary_lemma
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Spalten sortieren
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal, ['lemma_id','merkmal','is_lex','data_id','data','data_tag','lemma','lemma_tag','num','meta',])
wiktionary_merkmal.head(1)

### Grundformen anfügen

In [ ]:
# Grundformen anfügen (merkmal 'lemma')
spalten = ['lemma_id', 'lemma', 'tag_0', 'score' ]
grundformen = wiktionary_lemma[spalten].copy()
grundformen = pak.rename_col(grundformen, 'tag_0', 'lemma_tag')
grundformen['merkmal']  = 'lemma'
grundformen['is_lex']   = True
grundformen['data']     = grundformen.lemma.copy()
grundformen['data_id']  = grundformen.lemma_id.copy()
grundformen['data_tag'] = grundformen.lemma_tag.copy()
grundformen['meta']     = ''
grundformen['num']      = ''


# Spaltenvergleich
bpy.minivenn( grundformen.columns, wiktionary_merkmal.columns, format='print2')

wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, grundformen)
pak.sample(grundformen)

In [ ]:
# Sonderfall Mag.a löschen
mask = wiktionary_merkmal.data_id == 'Mag.a'
check_mask(wiktionary_merkmal,mask,0,2)

msg = 'Sonderfall Mag.a' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# Verwaiste Merkmale: Gibt es nicht, weil die abgetrennt oder gelöscht wurden
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#grid( wiktionary_merkmal, mask, )

# löschen
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, 'verwaist4')
grid(wiktionary_merkmal_trash, color='green')

In [ ]:
# fillna
wiktionary_merkmal['meta']          = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']      = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']     = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']           = wiktionary_merkmal.num.fillna('')
wiktionary_merkmal['merkmal_order'] = wiktionary_merkmal.merkmal_order.fillna(0)

In [ ]:
wiktionary_merkmal

In [ ]:
# update_col aus wiktionary_lemma
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag_0', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

In [ ]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
#pak.analyse_cols(wiktionary_merkmal)

### Leer oder ultrakurz

In [ ]:
# Prüfen: Keine Leerstrings
spalten = ['merkmal','data','lemma','lemma_tag']
for s in spalten:
    print(s)
    mask = wiktionary_merkmal[s] == ''
    check_mask( wiktionary_merkmal, mask, 0)

In [ ]:
# noSteig ermitteln und löschen
# data ist nur '-'
mask1 = wiktionary_merkmal.data.str.len() <= 1
mask2 = wiktionary_merkmal.lemma.str.len() > 1
mask3 = wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask4 = wiktionary_merkmal.is_lex == True
mask  =  mask1  &  mask2  &  mask3  &  mask4 
noSteig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 15369)

msg = 'noSteig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# noSteig in wiktionary_lemma kennzeichnen
mask1 =  wiktionary_lemma.lemma_id.isin(noSteig)
mask2 = ~wiktionary_lemma.member.str.contains('noSteig')
mask3 =  wiktionary_lemma.tagZ == 'ADJ'
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 146)

wiktionary_lemma.loc[mask,'member'] += ' noSteig' 
grid( wiktionary_lemma, mask )

In [ ]:
# Genus korrigieren
genus_m = ['Stromboli','Micky_Maus','Jesus_Christus','Jehova','Daniel_Düsentrieb','Dagobert_Duck','Tacken','Donald_Duck']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_m)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_m))

wiktionary_merkmal.loc[mask,'data'] = 'm'
wiktionary_merkmal[mask]

In [ ]:
# Genus korrigieren
genus_f = ['Arminia','Trauerwoche']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_f)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_f))

wiktionary_merkmal.loc[mask,'data'] = 'f'
wiktionary_merkmal[mask]

In [ ]:
# Genus n falsch
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 =  wiktionary_merkmal.data == 'n'
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['NN','NE'])
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0, 3)

msg = 'Genus n falsch' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# Ungültige Genus löschen
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 = ~wiktionary_merkmal.data.isin(['m','f','n'])
mask  =  mask1  &  mask2 
genus_ungültig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 725)

msg = 'Genus ungültig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 

mask = wiktionary_merkmal_trash.msg == msg
pak.analyse_freqs( wiktionary_merkmal_trash[mask],['data','lemma'])

In [ ]:
# genus_ungültig in wiktionary_lemma betrachten
# Sind da Kandidaten für manuelle Löschung oder Genuss setzen dabei?
mask1 =  wiktionary_lemma.lemma_id.isin(genus_ungültig)
mask2 = ~wiktionary_lemma.member.str.contains('noSg')
mask3 = ~wiktionary_lemma.member.str.contains('noPl')
mask4 = ~wiktionary_lemma.lemma_id.str.endswith('_V')
mask5 = ~wiktionary_lemma.member.str.contains('Person_vorname')
mask6 = ~wiktionary_lemma.member.str.contains('Geo')
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask( wiktionary_lemma, mask, 17)

spalten = ['lemma_id','tag','score','member']
a = wiktionary_lemma[mask][spalten].sort_values(['tag'])
grid(a)

In [ ]:
# data ist nur '-'
mask1 =  wiktionary_merkmal.data == '-'
mask2 =  wiktionary_merkmal.lemma_id != '-_M'
#mask3 = ~wiktionary_merkmal.lemma_tag.isin(['VVFIN'])
mask = mask1  &  mask2 # &  mask3
check_mask( wiktionary_merkmal, mask, 919)

msg = 'data ist nur -' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# superkurz >> alles bleibt wie es ist
mask1 =  wiktionary_merkmal.data.str.len() <= 1
mask2 =  wiktionary_merkmal.lemma.str.len() > 1   
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2  &  mask3     

#check_mask( wiktionary_merkmal, mask, 1292)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='superkurz' ) 

pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','data','lemma_id','lemma_tag'])
#grid( wiktionary_merkmal, mask )

### Allgemein

In [ ]:
# Explizit gesetzte Tags werden Merkmale
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data_tag != ''
mask3 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask4 = ~((wiktionary_merkmal.data_tag == 'ADJA')  &   (wiktionary_merkmal.lemma_tag == 'VVFIN')) # Partizipien
mask5 = wiktionary_merkmal.merkmal == 'lex'
mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal,mask,1,50)

# Merkmal erstellen
wiktionary_merkmal.loc[mask,'merkmal'] = wiktionary_merkmal[mask].data_tag

grid(wiktionary_merkmal,mask)
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
#grid(a)

In [ ]:
# manuelle Korrekturen
mask = wiktionary_merkmal.merkmal == 'genitiv Sg'
wiktionary_merkmal.loc[mask,'merkmal'] = 'Gen Sg'
mask = wiktionary_merkmal.merkmal ==         'Sg'
wiktionary_merkmal.loc[mask,'merkmal'] = 'Nom Sg'

In [ ]:
# Überblick wiktionary_merkmal
a = pak.analyse_freqs(wiktionary_merkmal, ['merkmal', 'data'] )
gridt(a)

In [ ]:
# alle Merkmale sind in plan_merkmal erfasst
mask = ~a.merkmal.isin(plan_merkmal.merkmal)
check_mask(a,mask,0)

In [ ]:
#grid(a,mask, color='green')

In [ ]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
search_str(wiktionary_merkmal[mask], suche_debug)

In [ ]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

### Wortarten -------

#### Partizipien 

In [ ]:
# Alle Adjektive
mask = wiktionary_lemma.tagZZ == 'A'
adj = wiktionary_lemma[mask].lemma
#adj

In [ ]:
# Alle Partizipien
mask = wiktionary_merkmal.merkmal == 'Ptz1'
ptz1 = wiktionary_merkmal[mask]
mask = wiktionary_merkmal.merkmal == 'Ptz2'
ptz2 = wiktionary_merkmal[mask].copy()
ptz_all = pak.add_rows(ptz1, ptz2)
#ptz_all

In [ ]:
# Alle Ptz1 sind auch als Adjektiv registriert
mask = ~ptz1.data.isin(adj)
check_mask(ptz1,mask,0)

In [ ]:
# Die meisten Ptz2 sind auch als Adjektiv registriert
mask = ~ptz2.data.isin(adj)
check_mask(ptz2,mask,0,120)
grid(ptz2,mask)

#### Verben

In [ ]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

In [ ]:
# Übersicht: Alle V-Tags in wiktionary_lemma
mask = wiktionary_lemma.tag.str.startswith('V')
a = pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma'])
grid(a)

In [ ]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

In [ ]:
# Infinitiv
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'lemma'
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 13605)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'INF'
grid(wiktionary_merkmal,mask)

In [ ]:
# Imperativ
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal.str.contains('Imp')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 36672)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'IMP'
grid(wiktionary_merkmal,mask)

In [ ]:
# Imperativ: VMIMP gibt es gar nicht
mask = wiktionary_merkmal.data_tag == 'VMIMP'
check_mask(wiktionary_merkmal,mask, 3)
wiktionary_merkmal.loc[mask,'data_tag'] = 'VVIMP'
#grid(wiktionary_merkmal,mask)

In [ ]:
# Partizip2
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz2' 
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,13885)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'PP'
grid(wiktionary_merkmal,mask)

In [ ]:
# Partizip1 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz1'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 26716)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
grid(wiktionary_merkmal,mask)

In [ ]:
# (1) Gerundivum: Schon fertig
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
grid(wiktionary_merkmal,mask)

In [ ]:
# Adjektivformen der Verben: ADJA ergänzen 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask3 = ~wiktionary_merkmal.data_tag.str.contains('ADJA')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal, mask, 27243)

adjaformen = wiktionary_merkmal[mask].copy()
adjaformen['data_tag'] = 'ADJA'
adjaformen['merkmal_order'] += 10

wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,adjaformen)
grid(adjaformen)

In [ ]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# Übersicht: Adjektivformen der Verben
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2','Gerundivum'])
mask  = mask1  &  mask2
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag','data'] )
a

In [ ]:
# VVIZU in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal != 'lemma'
mask4 = wiktionary_merkmal.data.str.replace('zu','') == wiktionary_merkmal.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask5 = wiktionary_merkmal.data.str.len() == wiktionary_merkmal.lemma.str.len() + 2
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal,mask, 8)

#wiktionary_merkmal.loc[mask,'data_tag']     = 'VVIZU'
#wiktionary_merkmal.loc[mask,'meta']    = '' 
#wiktionary_merkmal.loc[mask,'merkmal'] = 'Erweiterter_Infinitiv' 
grid(wiktionary_merkmal,mask)

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
# TODO_VVIZU
# VVIZU in translate_lex
mask1 = translate_lex.lemma_tag.str.startswith('V')
mask2 = translate_lex.data.str.replace('zu','') == translate_lex.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask3 = translate_lex.data.str.len() == translate_lex.lemma.str.len() + 2
mask  =  mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask, 5473)

vvizuformen = translate_lex[mask].copy()

In [ ]:
# vvizuformen anpassen und anfügen
vvizuformen = pak.drop_cols(vvizuformen,['lemma_tagZZ','lemma_lower','lemma_score'])
vvizuformen = pak.rename_col(vvizuformen,'member',    'meta'     )
vvizuformen = pak.rename_col(vvizuformen,'data_score', 'score'     )
vvizuformen['merkmal']       = 'VVIZU'
vvizuformen['is_lex']        = True
vvizuformen['data_tag']      = 'VVIZU'
vvizuformen['num']           = ''
vvizuformen['merkmal_order'] = 0

# Spaltenvergleich
bpy.minivenn( vvizuformen.columns, wiktionary_merkmal.columns, format='print2')

In [ ]:
# anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, vvizuformen)
grid(vvizuformen)

In [ ]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# Kontrolle: Keine anderen Infinitiv-like
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask4 = ~wiktionary_merkmal.merkmal.isin( ['lemma','Ptz2'] )
mask  = mask1  &  mask2  &  mask3  &  mask4

check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='Infinitiv-like' ) 
#grid(wiktionary_merkmal,mask)

In [ ]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

In [ ]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3

#grid(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag

In [ ]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

#### Nomen

In [ ]:
# Übersicht über alle explizit gesetzten tags (Tagquelle: lexeme_manuell)
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = wiktionary_merkmal.data_tag != ''
mask  = mask1  &  mask2  &  mask3
#grid(wiktionary_merkmal,mask)
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
(a)

In [ ]:
# Keine Doppeltags
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 =  wiktionary_merkmal.lemma_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

In [ ]:
# Irgendwelche unbekannten tags?
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = ~wiktionary_merkmal.data_tag.isin( ['NN','NE',''] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)
#grid(wiktionary_merkmal,mask)

In [ ]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 761247)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#grid(wiktionary_merkmal,mask)

In [ ]:
# Nomen analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask  = mask1  &  mask2  
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

#### Rest

In [ ]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.is_lex    == True
mask2 = wiktionary_merkmal.data_tag  == ''
mask3 = ~wiktionary_merkmal.lemma_tag.str[0].isin(['V','N'])
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 30194)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#a = pak.analyse_freqs( wiktionary_merkmal[mask], ['lemma_tag','data_tag'] )
#grid(a)

#### zweittag

In [ ]:
spalten = ['lemma_id','tag_1']
mask = wiktionary_lemma.tag.str.contains(' ')
check_mask( wiktionary_lemma, mask, 150)

zweitags = wiktionary_lemma[mask][spalten]

In [ ]:
# Merkmale anfügen mit zweittag
mask1 = wiktionary_merkmal.lemma_id.isin(zweitags.lemma_id)
mask2 = wiktionary_merkmal.lemma_tag == wiktionary_merkmal.data_tag
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 513)

zweitlexeme = wiktionary_merkmal[mask].copy()
zweitlexeme = pak.update_col(zweitlexeme, zweitags, on='lemma_id', col='tag_1', col_rename='data_tag')
zweitlexeme.meta = (zweitlexeme.meta + ' zweittag').str.strip()
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,zweitlexeme)
grid(zweitlexeme)

In [ ]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

#### Adjektive

In [ ]:
# Merkmal Positiv ist sinnlos
mask1 = wiktionary_merkmal.merkmal == 'Positiv'
mask2 = wiktionary_merkmal.data != wiktionary_merkmal.lemma
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0,10)
wiktionary_merkmal[mask]

In [ ]:
# Merkmal Positiv löschen
mask = wiktionary_merkmal.merkmal == 'Positiv'
check_mask( wiktionary_merkmal, mask, 13301)

msg = 'Merkmal Positiv ist sinnlos' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# Ist kein Adjektiv
mask1 =  wiktionary_merkmal.data_tag.isin(['ADJA','ADJD'])
mask2 = ~wiktionary_merkmal.merkmal.isin(['lemma','Ptz1','Ptz2','Gerundivum','Positiv','Komparativ','Superlativ','alt','alt_1','alt_2','lexAlt','lex','alt_abk'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, 16)

msg = 'Ist kein Adjektiv' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# Adjektiv-only Merkmale
mask1 = ~wiktionary_merkmal.data_tag.isin(['ADJA','ADJD','ADV'])
mask2 =  wiktionary_merkmal.merkmal.isin(['Gerundivum','Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, )

#a = wiktionary_merkmal[mask].sort_values(['data_tag','lemma_id'])
#grid(a)

msg = 'Adjektiv-only Merkmale' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

In [ ]:
# Steigerbare Adverbien #_TODO_ADJ_STEIGERBAR
mask1 =  wiktionary_merkmal.data_tag == 'ADV'
mask2 =  wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask4 = ~wiktionary_merkmal.lemma.isin(['sehr','gern','gerne','oft', 'bald','wohl'])
    
mask  = mask1  &  mask2  &  mask3  &  mask4 
check_mask( wiktionary_merkmal, mask, )

a = wiktionary_merkmal[mask].sort_values(['merkmal','data_tag','lemma_id'])
grid(a)

# 'sehr', 'mehr','am meisten'
# 'gern', 'lieber','am liebsten'
# 'bald', 'eher','am ehesten'
## 'oft',  'öfter','am öftesten'  # drin
## 'wohl', 'besser','am besten'   # drin

#msg = 'Irregeleitete Adjektive ' 
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
#grid( wiktionary_merkmal_trash, msg )

#### analysieren

In [ ]:
# Merkmale analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask  = mask1   
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

In [ ]:
# Keine Verwaisten lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='verwaist' ) 
#grid( wiktionary_merkmal, mask)

### Felder -------

#### data

In [ ]:
# Prüfen: Keine mehrfache Spaces
mask = wiktionary_merkmal.data.str.contains('  ')
check_mask(wiktionary_merkmal,mask,0)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

In [ ]:
# Prüfen: nicht leer
mask = wiktionary_merkmal.data.str.len() == 0
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='leer3' ) 

In [ ]:
# '⇀' ist nur in syllables
mask1 =  wiktionary_merkmal.data.str.contains('⇀')
mask2 = ~wiktionary_merkmal.merkmal.isin(['syllables'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Apostroph anschauen
mask =  wiktionary_merkmal.data.str.contains("'")
check_mask(wiktionary_merkmal,mask,40)
grid(wiktionary_merkmal,mask)

In [ ]:
# Komma anschauen
mask1 =  wiktionary_merkmal.data.str.contains(",")
mask2 = ~wiktionary_merkmal.lemma.str.contains(",")
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,33)
#grid(wiktionary_merkmal,mask)

#### num

In [ ]:
# num ist bei den meisten Lexemen sinnlos
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.num.str.len() > 0
#mask3 = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  #&  mask3

check_mask( wiktionary_merkmal, mask, 18770)
wiktionary_merkmal.loc[mask,'num'] = ''   
#grid(wiktionary_merkmal,mask, color='green')

In [ ]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['num','data',])

#### meta

In [ ]:
# remove_words
d = [ '(', ')', 'auch','nicht','bis','Adjektive','Adjektiv','Kombination','Substantive','Substantiv','allgemein','Adverbien','Verb','Adverbien','Adverb','andere','über']       
wiktionary_merkmal['meta'] = pak.remove_words( wiktionary_merkmal.meta, d )

In [ ]:
# replace_str
r = {       
    'seltener':'selten',
    'Schweiz':'schweizerisch',    
    'Österreich':'österreichisch', 
    'Süddeutschland':'süddeutsch',    
    'bairisch':'bayrisch',        
    'bairisch':'bayrisch',       
     
    }
wiktionary_merkmal['meta'] = pak.replace_str( wiktionary_merkmal.meta, r )

In [ ]:
# meta Überblick Lexeme
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
grid(a)

In [ ]:
# Seltene meta löschen
mask = a.meta_count < 20
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

In [ ]:
# meta Überblick Nichtlexeme
mask = wiktionary_merkmal.is_lex == False
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
grid(a)

In [ ]:
# Seltene meta löschen
mask = a.meta_count < 2
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == False
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

#### tag-Felder

In [ ]:
# data_tagZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZ', col_rename='data_tagZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZ', col_rename='data_tagZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZ','data_tag')

In [ ]:
# data_tagZZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZZ', col_rename='data_tagZZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='data_tagZZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZZ','data_tagZ')

In [ ]:
# Wieder löschen wenn kein Lexem
mask = wiktionary_merkmal.is_lex != True
wiktionary_merkmal.loc[mask,'data_tagZ']  = ''
wiktionary_merkmal.loc[mask,'data_tagZZ'] = ''

In [ ]:
# data_tagZ und data_tagZZ überall gesetzt
mask1  = wiktionary_merkmal.is_lex == True
mask2A = wiktionary_merkmal.data_tagZ  == ''
mask2B = wiktionary_merkmal.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(wiktionary_merkmal,mask, 0)

In [ ]:
# alle verwendeten tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['data_tag','merkmal'], splits=[' ', None] )
grid(a)

In [ ]:
# Gibt es unregistrierte tags?
alle_realen_tags = set(alle_tags) - {'AFIX','ZFIX','TODO','LEX_A','LEX_N', 'APPRART','MFIX','NNE','XY','_SP',}
seltsame_tags = set(a.data_tag) - alle_realen_tags
assert len(seltsame_tags) == 0

#mask = wiktionary_merkmal.tag.isin(seltsame_tags)
#wiktionary_merkmal[mask]

In [ ]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'TRUNC'
mask  =  mask1  &  mask2

grid(wiktionary_merkmal,mask, color='green')

In [ ]:
# Alle merkmale und ihre tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

In [ ]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'ADV'
mask3 =  wiktionary_merkmal.merkmal == 'Superlativ'
mask  =  mask1  &  mask2  &  mask3

grid(wiktionary_merkmal,mask, color='green')

In [ ]:
# lemma_tagZZ schreiben
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'lemma_tagZZ','lemma_tag')
wiktionary_merkmal['lemma_tagZZ'] = wiktionary_merkmal.lemma_tagZZ.fillna('')

In [ ]:
# check
mask1 = wiktionary_merkmal.lemma_tag != ''
mask2 = wiktionary_merkmal.lemma_tagZZ == ''
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask].head(1)

#### hunspell

In [ ]:
# Hunspell instance
from hunspell import Hunspell
hun = Hunspell('de_DE', hunspell_data_dir='/usr/share/hunspell') 

In [ ]:
# verify_hunspell
# liefert 1=verifiziert, 0=nicht verifiziert, -1=Besonderes Unicode-Zeichen
wiktionary_merkmal['hunspell'] = -2
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask] = wiktionary_merkmal[mask].apply(verify_hunspell, col_check='data', col_result='hunspell', hunspellinstance=hun, axis=1)
#wiktionary_merkmal = pak.change_datatype(wiktionary_merkmal, verbose=False)  

In [ ]:
# prüfen: alle erfasst?
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell < 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# abwerten
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell == 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'score'] *= 0.1
grid(wiktionary_merkmal,mask)

## translate_lex aufbauen ==========
* translate_lex enthält derzeit alle Wiktionary-Seitentitel, die keine Lemmas sind.
* Diverse Listen werden hier zusätzlich angefügt.
* Nach jeder Anfügung werden Dups entfernt.

In [ ]:
# set_translate_lex_score 
#from x32_Funktionen import *
translate_lex['score'] = 0
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

In [ ]:
#from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_lex)

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
# data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

assert translate_lex.data_id.is_unique

#a = pak.same_but_different(translate_lex,same='data_id',different='data')
#assert a.shape[0] == 0
#grid(a)

### translate_gm
* gm-Merkmale sind die Merkmale auf den Wiktionary-Seiten, die keine Lemmas sind. Also diejenigen, die unter wiktionary_nolemma bzw. translate_lex gespeichert sind.
* Daher weisen ihre lemma_id nicht auf ein Lemma, sondern auf ein Lexem. lemma_id und data_id müssen also hier vertauscht werden.

In [ ]:
# translate_lex 
# Dies sind die Seiten, deren Merkmale translate_gm enthält.
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
(search_str(translate_lex, ['besonne']))

In [ ]:
# lemma und data vertauschen
translate_gm = pak.rename_col(translate_gm, 'lemma_id',    'data_id_neu')
translate_gm = pak.rename_col(translate_gm, 'data_id',     'lemma_id')
translate_gm = pak.rename_col(translate_gm, 'data_id_neu', 'data_id')
translate_gm = pak.rename_col(translate_gm, 'data',        'lemma')

In [ ]:
# lemma_id nachschlagen (und dabei korrigieren!)
translate_gm = pak.update_col(translate_gm, translate_lex, on='data_id', col='lemma_id' ) 

In [ ]:
# translate_gm umformen
translate_gm = pak.rename_col(translate_gm, 'merkmal', 'member')
translate_gm['member'] = (translate_gm.member + ' ' + translate_gm.meta).str.strip()

translate_gm = pak.drop_cols(translate_gm, ['is_lex','data_tag','num','meta',])

In [ ]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

In [ ]:
from x32_Funktionen import *
translate_gm = pflege_translate_tabelle(translate_gm, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_gm)

In [ ]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

In [ ]:
assert translate_gm.lemma_home.nunique() == 2 # stand mal auf 1
assert translate_gm.data_home.nunique() == 2

In [ ]:
pak.analyse_freqs(translate_gm, ['data_home','lemma_home'])

In [ ]:
# translate_lex_trash pflegen
translate_lex_trash = pak.drop_cols(translate_lex_trash,['tag','tag_0','tag_1','tagZ','tagZZ'])
translate_lex_trash = pak.move_cols( translate_lex_trash, translate_lex.columns)
#translate_lex_trash

In [ ]:
pak.nnan(translate_gm)

In [ ]:
mask = translate_gm.lemma_tag.isnull()
check_mask(translate_gm,mask,0,100)

msg = 'verwaist_gm'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

In [ ]:
# Seltene Kombination (git's nicht mehr)
mask1 = translate_gm.lemma_home == 'T'
mask2 = translate_gm.data_home == 'T'
mask = mask1  &  mask2
check_mask(translate_gm,mask,0,51)

#grid(translate_gm,mask)
msg = 'Kombi TT'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

In [ ]:
# Bedeutungsloses löschen
mask1 = translate_gm.data == translate_gm.lemma
mask2A = translate_gm.data_tagZZ == translate_gm.lemma_tagZZ
mask2B = translate_gm.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_gm,mask,0,260)

translate_gm, df = pak.move_rows(translate_gm,mask)
grid(df)

In [ ]:
# Schrott rauslöschen
löschmich = ['Nominativ','Genitiv','Dativ','Akkusativ','Maskulinum','Femininum','Neutrum','Singular','Plural',
            'Hauptsatzkonjugation','Nebensatzkonjugation','Artikel','unbestimmt','Fugenelement','Allomorph',
             'Deklinationsendung','Komposita','Astronomischer Kalender','Konjugationsendung','Wortbildung',
             'am Lateinischen angelehnte Deklination','Hilfsverb','backen (regelmäßig)','Hauptsatz','Endung',
             'lateinisch','unbestimmter','Maskulinums','vernetzt','Derivatem','Ableitungsmorphem','griechisch',
             'Hapaxlegomenon','Interfix','Vollverb','unbestimmt','Artikel',
            ]
mask1A = translate_gm.lemma.isin(löschmich)
mask1B = translate_gm.data.isin(löschmich)
mask1C = translate_gm.lemma_id.isin(löschmich)
mask1D = translate_gm.data_id.isin(löschmich)
mask2 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask = (mask1A | mask1B | mask1C | mask1D)  &  mask2
check_mask(translate_gm,mask,10)

msg = 'Schrott1'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

In [ ]:
# Weiteren Schrott finden
mask1 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask2 = translate_gm.lemma_tagZZ != 'V'    # keine Verben
mask3 = ~translate_gm.lemma.str.endswith(('en','ern','eln','sein','tun')) # und sieht auch nicht aus wie ein Verb
mask = mask1  &  mask2  &  mask3
a = pak.analyse_freqs(translate_gm[mask],['lemma','data',])
grid(a)

In [ ]:
# suche_debug translate_gm
search_str(translate_gm, suche_debug)

In [ ]:
# not any_nan
assert not pak.any_nan(translate_gm)

In [ ]:
# Spalten anpassen
translate_gm = pak.drop_cols(translate_gm,['merkmal_order'])
translate_gm['score'] = 0

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_gm.columns, translate_lex.columns, format='print2')

In [ ]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_gm)

In [ ]:
# set_translate_lex_score
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
# nan entfernen
translate_lex['lemma_tag'] = translate_lex.lemma_tag.fillna('')

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id', 'data', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'score', ]
col_funcs   =  [ '',        '',     'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         '',          ''          ,  'sum',   ]   
col_names   =  [ '',        '',     'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', '',          ''          ,  'score', ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# data_id vorhanden und nicht widerspüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

### translate_m
* Alle Lexeme aus wiktionary_merkmal

In [ ]:
# Bei lemma: lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Alle Lexeme aus wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
translate_m = wiktionary_merkmal[mask].copy()
#translate_m

In [ ]:
# translate_m umformen
translate_m = pak.rename_col(translate_m, 'merkmal', 'member')
translate_m['member'] = ('merkmal ' + translate_m.member + ' ' + translate_m.meta).str.strip()

translate_m = pak.drop_cols(translate_m, ['is_lex','num','meta'])

In [ ]:
# lemma_tagZZ schreiben
translate_m = pak.update_col( translate_m, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
translate_m = pak.move_cols(  translate_m,'lemma_tagZZ','lemma_tag')
translate_m['lemma_tagZZ'] =  translate_m.lemma_tagZZ.fillna('')

In [ ]:
# temp
mask1 = translate_m.data  == 'ausgerutscht'
mask2 = translate_m.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_m[mask1]

In [ ]:
# data_id löschen, wenn nötig
mask1 = translate_m.data_id != translate_m.data
mask2 = translate_m.data_id == translate_m.lemma_id
mask = mask1  &  mask2
check_mask(translate_m,mask,25579)

translate_m.loc[mask,'data_id'] = ''
grid(translate_m,mask)
#a = pak.analyse_freqs(translate_m[mask],['member','data'])
#grid(a)

In [ ]:
# wiktionary_lemma suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
# data_id: Wenn leer, neu schreiben
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,25579)
translate_m[mask] = pak.update_col( translate_m[mask], wiktionary_lemma, cond='null', left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='score') 
translate_m[mask] = pak.update_col( translate_m[mask], translate_lex,    cond='null',      on=['data','lemma_tagZZ'],                             col='data_id',                        col_score='score') 

In [ ]:
# data_id neu erzeugen
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,0)

translate_m.loc[mask,'data_id'] = translate_m[mask].lemma + '~' + translate_m[mask].data + '~N'

In [ ]:
search_str(translate_m, suche_debug)

In [ ]:
#translate_m_bak = translate_m.copy()
#translate_m = translate_m_bak.copy()

In [ ]:
from x32_Funktionen import *
translate_m = pflege_translate_tabelle(translate_m, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_m)

In [ ]:
# Bedeutungsloses löschen
mask1 = ~translate_m.member.str.contains('lemma')
mask2 =  translate_m.data == translate_m.lemma
mask3A = translate_m.data_tagZZ == translate_m.lemma_tagZZ
mask3B = translate_m.data_tagZZ == ''
mask = mask1  &  mask2  &  (mask3A | mask3B)
#grid(translate_m,mask)
check_mask(translate_m,mask,357911)

translate_m, df = pak.move_rows(translate_m,mask)
grid(df)

In [ ]:
pak.analyse_freqs(translate_m, ['lemma_home','data_home'])

In [ ]:
# Neuzugänge
mask = translate_m.data_home == ''
grid(translate_m,mask)

In [ ]:
assert not pak.any_nan(translate_m)

In [ ]:
# Spalten anpassen
translate_m = pak.drop_cols(translate_m,['data_tagZ', 'hunspell', 'merkmal_order'])
translate_m['score'] = 0

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_m.columns, translate_lex.columns, format='print2')

In [ ]:
search_str(translate_m, suche_debug)

In [ ]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_m)

In [ ]:
# set_translate_lex_score
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
# nan entfernen
translate_lex['data_tag']   = translate_lex.data_tag.fillna('')
translate_lex['data_tagZZ'] = translate_lex.data_tagZZ.fillna('')

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'score', ]
col_funcs   =  [ '',        '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         '',          ''          ,  'sum',   ]   
col_names   =  [ '',        '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', '',          ''          ,  'score', ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
#pak.sample(translate_lex)

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

### translate_lma und translate_lmb
* aus wiktionary_lexmerkmal

In [ ]:
# zunächst wiktionary_lexmerkmal: Nichtlexeme löschen. Das sind vor allem syllables, ipa und rhymes.
mask = wiktionary_lexmerkmal.is_lex == False
check_mask( wiktionary_lexmerkmal, mask, 1523133) # 1523133

msg = 'Nichtlexeme'
wiktionary_lexmerkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_lexmerkmal, wiktionary_merkmal_trash, mask, msg )
#grid(wiktionary_merkmal_trash, msg)

In [ ]:
#pak.analyse_freqs(wiktionary_lexmerkmal,['merkmal','data'])

In [ ]:
# wiktionary_lexmerkmal
wiktionary_lexmerkmal.head(1)

In [ ]:
# lemma_id für data_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, on='data_id',  col='lemma_id')

In [ ]:
# translate_lma erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lma = wiktionary_lexmerkmal[mask].copy()

translate_lma = pak.rename_col(translate_lma, 'merkmal', 'member')
translate_lma['member'] = ('lma ' + translate_lma.member + ' ' + translate_lma.meta ).str.strip()

translate_lma = pak.drop_cols(translate_lma, ['lex_id','is_lex','data_tag','num','meta','data'])

In [ ]:
# lemma_id für lex_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, left_on='lex_id', right_on='data_id',  col='lemma_id', col_score='score')

In [ ]:
# translate_lmb erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lmb = wiktionary_lexmerkmal[mask].copy()

translate_lmb = pak.rename_col(translate_lmb, 'merkmal', 'member')
translate_lmb['member'] = ('lmb ' + translate_lmb.member + ' ' + translate_lmb.meta ).str.strip()

translate_lmb = pak.drop_cols(translate_lmb, ['data_id','data','data_tag','is_lex','num','meta','data'])

translate_lmb = pak.rename_col(translate_lmb, 'lex_id', 'data_id')

In [ ]:
# pflege_translate_tabelle
from x32_Funktionen import *
translate_lma = pflege_translate_tabelle(translate_lma, translate_tagZ, wiktionary_lemma, translate_lex )
translate_lmb = pflege_translate_tabelle(translate_lmb, translate_tagZ, wiktionary_lemma, translate_lex )

In [ ]:
# Bedeutungsloses löschen
mask1  = translate_lma.data       == translate_lma.lemma
mask2A = translate_lma.data_tagZZ == translate_lma.lemma_tagZZ
mask2B = translate_lma.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lma,mask,15885)

translate_lma, df = pak.move_rows(translate_lma,mask)
grid(df)

In [ ]:
# Bedeutungsloses löschen
mask1  = translate_lmb.data       == translate_lmb.lemma
mask2A = translate_lmb.data_tagZZ == translate_lmb.lemma_tagZZ
mask2B = translate_lmb.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lmb,mask,12)

translate_lmb, df = pak.move_rows(translate_lmb,mask)
grid(df)

In [ ]:
# Spalten anpassen
translate_lma = pak.drop_cols(translate_lma,['merkmal_order'])
translate_lma['score'] = 0

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_lma.columns, translate_lex.columns, format='print2')

In [ ]:
# Spalten anpassen
translate_lmb = pak.drop_cols(translate_lmb,['merkmal_order'])
translate_lmb['score'] = 0

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_lmb.columns, translate_lex.columns, format='print2')

In [ ]:
# Scores reduzieren
translate_lma['data_score'] /= 3
translate_lma['lemma_score'] /= 3
translate_lmb['data_score'] /= 3
translate_lmb['lemma_score'] /= 3

In [ ]:
#pak.sample(translate_lma)

In [ ]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_lma)
translate_lex = pak.add_rows(translate_lex,translate_lmb)

### pflegen

In [ ]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# verwaiste lemma
mask = translate_lex.lemma.isnull()
check_mask(translate_lex,mask,0,10)

msg = 'verwaist_lemma_10'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

In [ ]:
# verwaiste lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,100)

msg = 'verwaist_lemma_id_100'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

In [ ]:
#data_tag fillna
translate_lex['data_tag'] = translate_lex.data_tag.fillna('')

In [ ]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

In [ ]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )

### analyse_stringdiff

In [ ]:
from strsimpy.qgram import QGram 

qgram = QGram(2)
a = 'herbeispringen'
b = 'spring herbei'
c = 'herbeispringendes'
d = 'xfalsche Anfänger'
print(qgram.distance(a,b))
print(qgram.distance(a,c))
print(qgram.distance(a,d))

In [ ]:
from strsimpy.metric_lcs import MetricLCS
metric_lcs = MetricLCS()

print(metric_lcs.distance(a,b))
print(metric_lcs.distance(a,c))
print(metric_lcs.distance(a,d))

In [ ]:
# data_swaped: dächtest um >> umdächtest
translate_lex['data_swaped'] = translate_lex.data.str.split()
mask = translate_lex.data_swaped.str.len() == 2
translate_lex.loc[ mask, 'data_swaped'] = translate_lex[mask].data_swaped.str[-1] + translate_lex[mask].data_swaped.str[0]
translate_lex.loc[~mask, 'data_swaped'] = ''

In [ ]:
# Ermittelt die Differenz zwischen lex und lemma

def analyse_stringdiff(zeile):
    if len(zeile.data_swaped) > 0:
        result_a = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        result_b = metric_lcs.distance( zeile.data_swaped.lower(), zeile.lemma.lower() )     
        zeile['same_lexlemma'] = 1.0 - min(result_a, result_b)
    else:
        result   = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        zeile['same_lexlemma'] = 1.0 - result      
    return zeile

In [ ]:
# apply analyse_stringdiff 
translate_lex['same_lexlemma']   = 0.0
translate_lex = translate_lex.swifter.apply( analyse_stringdiff, axis=1 )
#translate_lex = pak.change_datatype(translate_lex, verbose=False)  

In [ ]:
# data_swaped wieder löschen
translate_lex = pak.drop_cols(translate_lex,'data_swaped')

In [ ]:
# same_lexlemma auf scores übertragen
translate_lex['data_score']  = translate_lex.data_score  * ((translate_lex.same_lexlemma + 0.1)**0.5)
translate_lex['lemma_score'] = translate_lex.lemma_score * ((translate_lex.same_lexlemma + 0.1)**0.5)

### data_id eindeutig

In [ ]:
# score komplett neu berechnen
translate_lex['score'] = 0
translate_lex = set_translate_lex_score(translate_lex, lexeme_manuell)

In [ ]:
#translate_lex_bak = translate_lex.copy()

# rückgängig
# translate_lex = translate_lex_bak.copy()

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ '',        '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         '',          ''          ,  'first',          'sum',   ]   
col_names   =  [ '',        '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', '',          ''          ,  'same_lexlemma',  'score', ] 

In [ ]:
# Gruppieren!
lookup_lex_tag = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

### weniger data_ids 

In [ ]:
translate_lex['data_id_bak'] = '' # Sicherheitskopie von data_id

In [ ]:
# Eine halbe Million data_ids, die nirgendwohin verweisen. 
# Sichern und Leeren!
mask1 =  translate_lex.data != ''
mask2 = ~translate_lex.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~translate_lex.data_id.isin(wiktionary_merkmal.data_id)
mask = mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask,455800)

# kennzeichnen
translate_lex.loc[mask,'data_id_bak'] = translate_lex[mask].data_id  # sichern
translate_lex.loc[mask,'data_id']     = ''                           # leeren

#grid(translate_lex[mask],pak.sample )

In [ ]:
# Sicherheitskopie
#translate_lex_bak = translate_lex.copy()
#translate_lex = translate_lex_bak.copy()

In [ ]:
# Zweites data_id-Feld
translate_lex['data_id2'] = translate_lex.data_id.copy()
translate_lex = pak.move_cols(translate_lex,'data_id2','data_id')
#translate_lex

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id',  'data_id2', 'data_id_bak', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ 'max',      'min',      'max',         '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         'first',     'first',       'first',          'sum',   ]   
col_names   =  [ 'data_id',  'data_id2', 'data_id_bak', '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# Beide data_id leer? Dann data_id_bak verwenden
mask1 = translate_lex.data_id == ''
mask2 = translate_lex.data_id2 == ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,354478)
translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id_bak
#grid(translate_lex,mask)

In [ ]:
# Prüfen: data_id immer belegt
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

In [ ]:
# data_ids sind verschieden, aber nahezu gleich
# z.B. abstoßend_1 und abstoßend
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask3 = pak.remove_str( translate_lex.data_id, ['_1','_2','_3','_4','_MADJ',], safemode=False )  ==  pak.remove_str( translate_lex.data_id2, ['_1','_2','_3','_4','_MADJ',], safemode=False )
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask,393)

translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id2 # das Minimum, also die kürzere id
translate_lex.loc[mask,'data_id2'] = ''
#grid(translate_lex,mask)

In [ ]:
# Jetzt gibt es keine widersprüchliche data_id mehr
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask,)

In [ ]:
# Hilfsspalten löschen
translate_lex = pak.drop_cols(translate_lex,['data_id2','data_id_bak'])

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
# pflege_translate_tabelle
translate_lex_ranked  = pak.rank(translate_lex, col_score='score', cols_group='data_id', on_conflict='first')
translate_lex         = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex_ranked )

In [ ]:
# translate_lex: data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
assert not pak.any_nan(translate_lex)

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'ausführt'
translate_lex[mask]

In [ ]:
# Beispiel anzeigen
#finde = ['eine','einer','eines','einen','ein','einem']
#mask = translate_lex.data.isin(finde)
#translate_lex[mask]

In [ ]:
#pak.analyse_freqs(translate_lex,['data_home','lemma_home'])

## wiktionary_merkmal

In [ ]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Sicherheitskopie
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()
#wiktionary_merkmal = wiktionary_merkmal_bak.copy()

In [ ]:
wiktionary_merkmal['data_id_bak'] = '' # Sicherheitskopie von data_id

In [ ]:
translate_lex

In [ ]:
# verwaiste data_id:
# leeren und neu belegen
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(wiktionary_merkmal,mask, 0, 34)

# kennzeichnen
wiktionary_merkmal.loc[mask,'data_id_bak'] = wiktionary_merkmal[mask].data_id  # sichern
wiktionary_merkmal.loc[mask,'data_id']     = ''                                # leeren

# data_id schreiben 
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], wiktionary_lemma, cond='null', left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='score')
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], translate_lex,    cond='null',      on=['data','lemma_tagZZ'],                             col='data_id',  col_score='score', )

# Kontrolle
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_merkmal,mask, 0)

# anschauen
mask = wiktionary_merkmal.data_id_bak != ''
grid(wiktionary_merkmal,mask)

In [ ]:
mask = wiktionary_merkmal.data_id == 'Petersen~Peterson'
wiktionary_merkmal[mask]

In [ ]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Verlinkung auf data prüfen 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, on='data_id', col='data', col_rename='data_check', col_score='score')
mask_kaputt = wiktionary_merkmal.data != wiktionary_merkmal.data_check
check_mask(wiktionary_merkmal,mask_kaputt,0)
#grid(wiktionary_merkmal,mask_kaputt)

In [ ]:
# meta == lemma
mask = wiktionary_merkmal.meta == wiktionary_merkmal.lemma
check_mask(wiktionary_merkmal,mask,57940)

wiktionary_merkmal.loc[mask,'meta'] = ''

In [ ]:
# meta == lex 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, left_on='meta', right_on='data', col='lemma', col_rename='löschmich_lemma', col_score='score')

mask = wiktionary_merkmal.lemma == wiktionary_merkmal.löschmich_lemma
check_mask(wiktionary_merkmal,mask,13845)
wiktionary_merkmal.loc[mask,'meta'] = ''

wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,'löschmich_lemma')

In [ ]:
# Hilfsdaten löschen
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['data_id_bak','notiz','data_check'])

In [ ]:
grid(wiktionary_merkmal,pak.sample)

In [ ]:
#mask = translate_lex.data.isin(['besonnte','überlegte'])
#grid(translate_lex,mask)

In [ ]:
#translate_lex_bak = translate_lex.copy()

# rückgängig
# translate_lex = translate_lex_bak.copy()

In [ ]:
raise Stop

### Kontrolle wiktionary_merkmal (aus d01)
* Diverse Kontrollen

In [ ]:
# plan_merkmal Steuertabelle neu erstellen
try:
    del plan_merkmal
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
plan_merkmal = plan_merkmal_erstellen()
#pak.sample(plan_merkmal)
grid(plan_merkmal)

In [ ]:
# Merkmale lt. Steuertabelle und lt. Realität
mask = (plan_merkmal.plan != 2)
merkmale_plan = set(plan_merkmal[mask].merkmal)
merkmale_data = set(wiktionary_merkmal.merkmal.unique())

In [ ]:
# left_only:  Merkmale, die in er Realität nicht auftreten
# right_only: Merkmale, die nicht geplant sind
bpy.minivenn(merkmale_plan,merkmale_data,format='print2')

In [ ]:
# Fast alle in der Steuertabelle plan_merkmal erfassten Merkmale kommen in wiktionary_merkmal tatsächlich vor.
ausnahmen = {'Gerundivum', 'Konj1', 'Imp', 'lemma', 'übersicht', 'down_part', 'Konj', 'Konj2', 'PTKVZ'}
fehlende_merkmale = bpy.minivenn(merkmale_plan - ausnahmen,merkmale_data,format='dict')['left_only']
assert len(fehlende_merkmale) == 0

In [ ]:
# Tauchen die Ausnahmen in wiktionary_merkmal_trash auf?
# Sollten sie nicht.
mask1 =  wiktionary_merkmal_trash.merkmal.isin(  ausnahmen - {'nurWortart'}  )
mask2 = ~wiktionary_merkmal_trash.merkmal.isin(['übersicht'])
mask = mask1  & mask2
check_mask(wiktionary_merkmal_trash,mask,0)
#wiktionary_merkmal_trash[mask]

In [ ]:
raise Stop

In [ ]:
# TODO_TAGS_IN_MERKMALE
bgc('orange')
mask = wiktionary_merkmal.merkmal.isin({'VAFIN', 'VMFIN', 'VVIMP', 'VVIZU',})
wiktionary_merkmal[mask]

In [ ]:
# alle Merkmale in wiktionary_merkmal sind in der Steuertabelle plan_merkmal erfasst
ausnahmen = {'VAFIN', 'VMFIN', 'VVIMP', 'VVIZU', 'ADV'} ## TODO_TAGS_IN_MERKMALE eigentlich sollte es keine Ausnahmen geben!!
unregistrierte_merkmale = bpy.minivenn(merkmale_plan,merkmale_data - ausnahmen,format='dict')['right_only']
print(unregistrierte_merkmale)

In [ ]:
# Probleme anschauen
mask = wiktionary_merkmal.merkmal.isin(unregistrierte_merkmale)
grid(wiktionary_merkmal, mask, color='green')

In [ ]:
#assert len(unregistrierte_merkmale) == 0

In [ ]:
# TODO_unregistrierte_merkmale
bgc('orange')
mask = wiktionary_merkmal.merkmal.isin(unregistrierte_merkmale)
grid(wiktionary_merkmal, mask, color='green')

In [ ]:
# Keine nan in data
assert pak.nnan(wiktionary_merkmal.data) == 0

In [ ]:
# Eindeutige Datentypen
for col in wiktionary_merkmal.columns:
    print(col)
    if col == 'num':
        continue
    assert pak.ntypes(wiktionary_merkmal[col]) == 1

In [ ]:
wiktionary_merkmal.shape

## translate_lemma ==========
* Datensätze aus translate_lex, deren IDs von Lemma zu Lemma zeigen
* Aber keine Lemmas (also keine Verweise auf sich selbst)
* Datensätze aus translate_abk

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

In [ ]:
# translate_lemma erzeugen
keine_lemmas = ['einen','beigen','Montage',]

mask1 =  translate_lex.data_home == 'B'             # beides sind lemmas
mask2 =  translate_lex.data != translate_lex.lemma  # sie sind verschieden
mask3 = ~translate_lex.data.isin(keine_lemmas)      # gehören nicht zu den Ausnahmen
mask4 = ~translate_lex.member.str.contains('Ptz1')  # kein Partizip1
mask5 = ~translate_lex.member.str.contains('Ptz2')  # kein Partizip2
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lex,mask,57729)
#grid(translate_lex,mask)
translate_lex, translate_lemma = pak.move_rows(translate_lex,mask, msg=None)

In [ ]:
pak.sample(translate_lemma)

### translate_abk

In [ ]:
# Alle Kurzformen sind in wiktionary_lemma
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')

mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask,0,50)

msg = 'kurz_id inzwischen verwaist'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg )
grid(translate_abk_trash,msg)

In [ ]:
assert not pak.any_nan(translate_abk)

In [ ]:
# Umbenennungen. data und lemma sind nicht passende Bezeichnungen, aber pflege_translate_tabelle wünscht diese. 
# Wird später durch passendere ersetzt.
spalten = ['kurz_id','lang_id','todo']
df = translate_abk[spalten].copy()
df = pak.rename_col(df, 'kurz_id', 'data_id')
df = pak.rename_col(df, 'lang_id', 'lemma_id') 
df = pak.rename_col(df, 'todo', 'member') 
df['member'] = ('abk ' + df.member).str.strip()
df.head(1)

In [ ]:
# lang_id war kein Lemma >> translate_lex nachschlagen
df['lemma_id_new'] = ''
mask = ~df.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(df,mask,30)

df.loc[mask] = pak.update_col( df[mask],          
                               translate_lex,
                               col_score='score', 
                               left_on='lemma_id',
                               right_on='data_id',
                               col='lemma_id',
                               col_rename='lemma_id_new',
                               
                       )   

mask = df.lemma_id_new != ''
df.loc[mask,'lemma_id'] = df[mask].lemma_id_new

df = pak.drop_cols(df, 'lemma_id_new')
#df[mask]

In [ ]:
# Müll löschen
löschmich = ['PLURV','PIMS','MRSA','EBRG','BodSchätzDB','ZÄPrO','SJZ','Mmes.','RÄ','GDPdU','E_570','E_163']
mask = df.data_id.isin(löschmich)
df = pak.drop_rows(df,mask)

In [ ]:
# dups löschen
mask = df.duplicated(subset=['data_id','lemma_id'])
df = pak.drop_rows(df,mask)

In [ ]:
assert not pak.any_nan(df)

In [ ]:
pak.nnan(df)

In [ ]:
# score wird später zu data_score
df = pak.update_col(df,          
                    wiktionary_lemma,
                    left_on='data_id',
                    right_on='lemma_id',
                    col='score',
                       )   

In [ ]:
# pflege_translate_tabelle
df = pflege_translate_tabelle(df, translate_tagZ, wiktionary_lemma, translate_lex )

In [ ]:
# data_swaped: dächtest um >> umdächtest
df['data_swaped'] = df.data.str.split()
mask = df.data_swaped.str.len() == 2
check_mask(df,mask)
df.loc[ mask, 'data_swaped'] = df[mask].data_swaped.str[-1] + df[mask].data_swaped.str[0]
df.loc[~mask, 'data_swaped'] = ''

In [ ]:
#pak.analyse_cols(df)

In [ ]:
# apply analyse_stringdiff 
df['same_lexlemma']   = 0.0
df = df.apply( analyse_stringdiff, axis=1 )

In [ ]:
# data_swaped wieder löschen
df = pak.drop_cols(df,'data_swaped')

In [ ]:
# set_translate_lex_score
df['score'] = 0
df = set_translate_lex_score(df, lexeme_manuell)

In [ ]:
# data_tag
df['data_tag'] = df.lemma_tag.copy()

In [ ]:
# Spaltenvergleich
bpy.minivenn( df.columns, translate_lemma.columns, format='print2')

In [ ]:
# add_rows
translate_lemma = pak.add_rows(translate_lemma, df)

In [ ]:
# rename_col
translate_lemma = pak.rename_col(translate_lemma, 'data_id',     'lem_id')
translate_lemma = pak.rename_col(translate_lemma, 'data_home',   'lem_home')
translate_lemma = pak.rename_col(translate_lemma, 'data',        'lem')
translate_lemma = pak.rename_col(translate_lemma, 'data_score',  'lem_score')

In [ ]:
translate_lemma = pak.rename_col(translate_lemma, 'data_tag',  'lem_tag')
translate_lemma = pak.rename_col(translate_lemma, 'data_tagZZ','lem_tagZZ')

In [ ]:
grid(translate_lemma, pak.sample)

In [ ]:
# lem_tag
translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tag',
                                  col_rename = 'lem_tag',
                       )   

translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tagZZ',
                                  col_rename = 'lem_tagZZ',
                       )   

translate_lemma['lem_tag']   = translate_lemma.lem_tag.fillna('')
translate_lemma['lem_tagZZ'] = translate_lemma.lem_tagZZ.fillna('')
#translate_lemma = pak.move_cols( translate_lemma, ['lem_tag','lem_tagZZ'],'lem')      

### belöschen

In [ ]:
# translate_lemma_trash
translate_lemma_trash = translate_lemma.head(0)
translate_lemma_trash['msg'] = ''

In [ ]:
a = pak.analyse_freqs(translate_lemma,['lemma','lem'])
grid(a)

In [ ]:
#translate_lemma

In [ ]:
# aussortiert1
erlaubt = ['H2O','CAD','CH','GAU','DIN','UN','UNO','LCD','IGS','2D','3D','4D','ISP','IR','DRK','MPU','OOP','ÖPNV','° C','°C','HDTV','° F','°F']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 =  translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask3 =  translate_lemma.lem.str.len() <= 4
mask4 =  translate_lemma.lem.str.isupper()
mask5 = ~translate_lemma.lem.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lemma,mask,0,215)

msg = 'aussortiert1'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

In [ ]:
# aussortiert2
# 

löschmich = ['-','Hag','DrIng','Deik','Fritsch','Frerk', 'Fricke','nomen nominandum',
             'wullacken','Diastema mediale','BM', 'JO', 'NB',
             'Rosch ha-Schana','Wiktionary','Fuzo', 'Fuzo.',
             'Intern', 'Intern.', 'iNet', 'iNet.', 'ZUM', 'ut supra','Art', 'GSSD', 
             'Met','DE', 'DFV', 'DHB', 'DSV','AN', 'AS', 'AUR','GVBl', 'GWB', 'GenG', 'Ges',
             'BKG', 'BVT', 'BfN', 'BfS','ritenuto','Folio','Aserbaidschan-Manat',
             'sensu lato','nes.','ceterum censeo',
             'pro rata','GGVS','IRPA','Tau','ABC',
             'ISA','DOS','ABF','ÖFB','VOC','KPÖ','IN','ZIF','PVP','BDO','g. e.','AUT','EUV',
            ]
behalten  = ['nen','ne','nes', 'geeint','einend', ]
mask1A =  translate_lemma.lem.isin(löschmich)
mask1B =  translate_lemma.lemma.isin(löschmich)
mask2  = ~translate_lemma.lem.isin(behalten)
mask = (mask1A | mask1B)  &  mask2
check_mask(translate_lemma,mask,150)

msg = 'aussortiert2'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

In [ ]:
# unabhängige_lexeme

unabhängige_lexeme  = ['Alu','ALU','Arbeitslosenunterstützung','CO2', 'CO₂','chemisch','USK','vergl', 'verglasen','vergl.','24/7','Stunde','Schriftwerk','Buch','und','E 962','Verbindung',
                       'km/h','Kilometer','weiblich','männlich','Elefant','frühneuhochdeutsch','sein','Disko','modern','tun','Handlung',
                      ]
mask1 = translate_lemma.lem.isin(unabhängige_lexeme)
mask2 = translate_lemma.lemma.isin(unabhängige_lexeme)
mask = mask1  &  mask2
check_mask(translate_lemma,mask,17)

msg = 'unabhängige_lexeme'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

In [ ]:
# Mögliche Probleme
# 
spalten = ['lem','lemma','member','lemma_score']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 = translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask = mask1  &  mask2
a = translate_lemma[mask][spalten].sort_values(['lemma_score'], ascending=[False])
grid(a)

In [ ]:
# lem_tag übertragen, falls nötig
mask = translate_lemma.lem_tag.isnull()
check_mask(translate_lemma,mask,0,10)

translate_lemma.loc[mask,'lem_tag']   = translate_lemma[mask].lemma_tag
translate_lemma.loc[mask,'lem_tagZZ'] = translate_lemma[mask].lemma_tagZZ

In [ ]:
# alle getaggt
maskA = translate_lemma.lem_tag == ''
maskB = translate_lemma.lem_tagZZ == ''
mask = maskA | maskB
check_mask(translate_lemma,mask,0)

In [ ]:
mask = translate_lemma.isna().any(axis=1)
translate_lemma[mask]

In [ ]:
#assert not pak.any_nan(translate_lemma)
pak.nnan(translate_lemma)

In [ ]:
pak.sample(translate_lemma)

## wiktionary_merkmal_text ==========

In [ ]:
# lemma übertragen
assert not pak.any_nan(wiktionary_lemma, without=['page_id','page_part','data_id',])
wiktionary_merkmal_text = pak.update_col( wiktionary_merkmal_text, wiktionary_lemma, on='section_id2', col='lemma'  )

In [ ]:
# verwaise_section_id2 finden
mask = wiktionary_merkmal_text.lemma.isnull()
verwaise_section_id2 = wiktionary_merkmal_text[mask].section_id2.drop_duplicates()
verwaise_section_id2

In [ ]:
bgc('orange') ## TODO_verwaiste_lemmas
mask1 =  wiktionary_merkmal_text.section_id2.isin(verwaise_section_id2)
mask2 = ~wiktionary_merkmal_text.section_id2.isin(wiktionary_trash.section_id2)
mask = mask1  &  mask2
grid(wiktionary_merkmal_text[mask].sort_values('section_id2'))

In [ ]:
# verwaiste Datensätze anschauen
mask = wiktionary_merkmal_text.lemma.isnull()
grid(wiktionary_merkmal_text,mask)

In [ ]:
raise Stop

In [ ]:
# verwaiste Datensätze löschen
mask = wiktionary_merkmal_text.lemma.isnull()
check_mask(wiktionary_merkmal_text,mask,1217)
wiktionary_merkmal_text = pak.drop_rows(wiktionary_merkmal_text,mask)

In [ ]:
# suche_debug

mask = wiktionary_merkmal_text.lemma_id.isin(suche_debug)
grid(wiktionary_merkmal_text,mask)
#wiktionary_lemma[mask].lemma

In [ ]:
# Striche entfernen, auch in wiktionary_merkmal_text
mask1 = wiktionary_merkmal_text.data.str.count("\'") > 1
mask2 = wiktionary_merkmal_text.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal_text,mask,2290)
wiktionary_merkmal_text.loc[mask,'data'] = wiktionary_merkmal_text[mask].data.str.replace("'", '')
#grid( wiktionary_merkmal_text, mask )

In [ ]:
# not any_nan
#wiktionary_merkmal_text['merkmal_order2'] = wiktionary_merkmal_text.merkmal_order2.fillna(0) 
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.analyse_cols(wiktionary_merkmal_text)

In [ ]:
# Verwaiste löschen
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,8263)
#grid(wiktionary_merkmal_text,mask)
wiktionary_merkmal_text = pak.drop_rows(wiktionary_merkmal_text,mask)

In [ ]:
# lemma übertragen
wiktionary_merkmal_text = pak.update_col( wiktionary_merkmal_text, wiktionary_lemma, on='lemma_id', col='lemma'  )

In [ ]:
# meta == lemma
mask = wiktionary_merkmal_text.meta == wiktionary_merkmal_text.lemma
check_mask(wiktionary_merkmal_text,mask,204089)

wiktionary_merkmal_text.loc[mask,'meta'] = ''

In [ ]:
# data leer
mask = wiktionary_merkmal_text.data == ''
check_mask(wiktionary_merkmal_text,mask,27)

wiktionary_merkmal_text = pak.drop_rows(wiktionary_merkmal_text,mask)

In [ ]:
# suche_debug
mask = wiktionary_merkmal_text.lemma.isin(suche_debug)
grid(wiktionary_merkmal_text,mask)
#wiktionary_lemma[mask].lemma

In [ ]:
#a = pak.analyse_cols(wiktionary_merkmal_text)
#(a)

In [ ]:
# Was ist drin?
a = pak.analyse_freqs(wiktionary_merkmal_text,['merkmal','data'] )
grid(a)

In [ ]:
# TODO
mask1 = wiktionary_merkmal_text.merkmal == 'bsp_re'
mask2 = wiktionary_merkmal_text.data.str.contains('-')
mask  = mask1  &  mask2
grid( wiktionary_merkmal_text, mask )

## wiktionary_lemma =====

In [ ]:
# Anzahl der Lexeme untersuchen

mask = wiktionary_merkmal.is_lex == True
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'lex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['lex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='lex_anz')

In [ ]:
# Anzahl der Nichtlexeme untersuchen

mask = wiktionary_merkmal.is_lex == False
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'nolex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['nolex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='nolex_anz')

In [ ]:
# Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
grid(wiktionary_lemma,mask, color='green')

In [ ]:
raise Stop

In [ ]:
member überprüfen wie in d01

## wiktionary_merkmal ==========

In [ ]:
# verwaiste wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask, 0)

In [ ]:
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])
#grid(a)

In [ ]:
# Fehlende data_id?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2 
check_mask(wiktionary_merkmal,mask,0, 10)

### Überschneidungen
* Lexeme, die sich in translate_lex finden aber auch direkt in wiktionary_lemma >> entweder oder!

In [ ]:
# mask_LL erstellen
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1550)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

In [ ]:
# wiktionary_lemma belöschen: Abkürzungen wieder löschen
mask1 =  wiktionary_lemma.lemma.isin(translate_lex[mask_LL].data)
mask2 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask3 = ~wiktionary_lemma.member.str.contains('manuell')
mask4 =  wiktionary_lemma.page_part == -1
#mask5 = ~wiktionary_lemma.lemma.isin(['pos',])
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_lemma, mask, 0, 150)

msg = 'Überschneidungen_1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

In [ ]:
# wiktionary_merkmal: verwaiste_Lemmas löschen
mask1 =  wiktionary_merkmal.merkmal == 'lemma'
mask2 =  wiktionary_merkmal.data_id != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3 &  mask4
check_mask(wiktionary_merkmal,mask,0,150)

msg = 'verwaiste_Lemmas'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,150)
# translate_lemma[mask]

msg = 'verwaist_lem_id'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

In [ ]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

#msg = 'verwaist_lemma_id'
#translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
#grid(translate_lemma_trash,msg)

In [ ]:
# Löschen: Einige sonstige Verwaisungen
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,30)
#grid(wiktionary_merkmal,mask)

msg = 'verwaist_data_id_24'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# TODO Ungelöste Probleme 
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1400)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

## translate_lex.data_tag 

In [ ]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask1]

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

In [ ]:
# Beispiel anzeigen
mask = translate_lemma.lem == 'Ackermann'
translate_lemma[mask]

In [ ]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex   = pflege_translate_tabelle(translate_lex,   translate_tagZ, wiktionary_lemma)
translate_lemma = pflege_translate_tabelle(translate_lemma, translate_tagZ, wiktionary_lemma, translate_lex )

In [ ]:
# Nachschlagetabelle vorbereiten
spalten = ['data_id','data','data_tag','lemma_id','lemma','lemma_tag','lemma_tagZZ']
mask = wiktionary_merkmal.is_lex == True
df = wiktionary_merkmal[mask][spalten].drop_duplicates().sort_values(spalten)
#df

In [ ]:
# Prüfe Anzahl data_tag
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

In [ ]:
# Vorhandene data_tag übertragen
# sonst leeren
translate_lex['data_tag_0'] = translate_lex.data_tag
translate_lex['data_tag_1'] = translate_lex.data_tag
translate_lex = pak.move_cols(translate_lex, ['data_tag','data_tag_0','data_tag_1'],'data')

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

In [ ]:
# data_tag übertragen 
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
           
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
  
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

print()
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
   
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

In [ ]:
# data_tag fehlt
maskA = translate_lex.data_tag_0 == ''
maskB = translate_lex.data_tag_1 == ''
mask = maskA | maskB
check_mask(translate_lex,mask,356699)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

In [ ]:
# data_tag eindeutig
mask = translate_lex.data_tag_0 == translate_lex.data_tag_1
check_mask(translate_lex,mask,391328)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

In [ ]:
# data_tag uneindeutig
mask1 = translate_lex.data_tag_0 != translate_lex.data_tag_1
mask2 = translate_lex.data_tag_0 != ''
mask3 = translate_lex.data_tag_1 != ''
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask, 22364)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0 + ' ' + translate_lex[mask].data_tag_1
#grid(translate_lex,mask)

In [ ]:
# Sicherstellen: data_tag fehlt nirgens komplett
mask = translate_lex.data_tag == ''
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask)

In [ ]:
# nach Priorität sortieren (laut Steuertabellen)
translate_lex['data_tag'] = tag_sortieren(translate_lex.data_tag)

In [ ]:
# translate_lex vorbereiten
translate_lex = pak.drop_cols(translate_lex,['data_tag_0','data_tag_1','data_tagZZ'])    
translate_lex = pak.rename_col(translate_lex,'data_tag','tag')    

In [ ]:
# pflege_tag_felder
translate_lex = pflege_tag_felder(translate_lex, translate_tagZ)

In [ ]:
# # translate_lex nachbereiten
translate_lex = pak.rename_col(translate_lex,'tag',   'data_tag')    
translate_lex = pak.rename_col(translate_lex,'tag_0', 'data_tag_0')   
translate_lex = pak.rename_col(translate_lex,'tag_1', 'data_tag_1')   
translate_lex = pak.rename_col(translate_lex,'tagZZ', 'data_tagZZ')   
translate_lex = pak.drop_cols(translate_lex,['tagZ'])    
grid(translate_lex,pak.sample)

In [ ]:
mask = translate_lex.data == 'vervollständigt'
translate_lex[mask]

In [ ]:
#mask = wiktionary_lemma.tag.str.contains(' ')
#wiktionary_lemma[mask]

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'lachen'
translate_lex[mask]

In [ ]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask]

## Prüfen ==========

### Unique

In [ ]:
# wiktionary_lemma: lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

In [ ]:
# wiktionary_merkmal: data_id vorhanden und nicht widersprüchlich?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

# Vergebene data_id nicht widersprüchlich?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# translate_lex: data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

### Mögliche Probleme

In [ ]:
# Mögliche Probleme
spalten = ['data','lemma','lemma_score']
mask =  translate_lex['same_lexlemma'] < 0.3
a = translate_lex[mask][spalten].sort_values(['lemma_score'], ascending=[False])
grid(a)

In [ ]:
mask = translate_lex.same_lexlemma < 0.5
a = pak.analyse_freqs(translate_lex[mask],['lemma','data'])
grid(a)

### Verwaisungen

In [ ]:
# translate_lex lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,140)

msg = 'verwaist_lemma_id_endkontrolle'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

In [ ]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,32)
# translate_lemma[mask]

msg = 'verwaist_lem_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

In [ ]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

msg = 'verwaist_lemma_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

In [ ]:
# wiktionary_lemma.data_id (nur leeren)
mask1 =  wiktionary_lemma.data_id != ''
mask2 = ~wiktionary_lemma.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_lemma.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,1)

wiktionary_lemma.loc[mask,'data_id'] = ''
grid(wiktionary_lemma,mask)

In [ ]:
# wiktionary_merkmal lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0,140)
# wiktionary_merkmal[mask]

msg = 'verwaist_lemma_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# verwaist wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask4 = wiktionary_merkmal.data_id.isin(translate_lex_trash.data_id)
mask = mask1  &  mask2  &  mask3 # &  mask4
check_mask(wiktionary_merkmal,mask)

#grid(wiktionary_merkmal,mask)

In [ ]:
# wiktionary_merkmal data_id
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,40)

msg = 'verwaist_data_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
#grid(wiktionary_merkmal,mask)

In [ ]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])

In [ ]:
# wiktionary_merkmal_text lemma_id
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,0)
# wiktionary_merkmal_text[mask]

#msg = 'verwaist_lemma_id_endkontrolle'
#wiktionary_merkmal_text, df = pak.move_rows( wiktionary_merkmal_text, mask, msg )
#grid(df)

### NaN

In [ ]:
assert not pak.any_nan(wiktionary_lemma)
#pak.nnan(wiktionary_lemma)

In [ ]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

In [ ]:
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.nnan(wiktionary_merkmal_text)

In [ ]:
assert not pak.any_nan(translate_lemma)
#pak.nnan(translate_lemma)

In [ ]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

### Div

In [ ]:
# lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# lemma und data müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma != wiktionary_merkmal.data
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

### Partizipien prüfen

In [ ]:
#suche_L = 'missraten' 
#suche_P = 'missraten'  

suche_L = 'gefriertrocknen' 
suche_P = 'gefriergetrocknet'  

In [ ]:
# wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data  == suche_P
mask = mask1  &  mask2
wiktionary_merkmal[mask]

In [ ]:
# das Verb in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_L
mask2 = wiktionary_lemma.tagZZ == 'V'
mask = mask1  &  mask2

suche_L_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

In [ ]:
# das Adjektiv in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_P
mask2 = wiktionary_lemma.tagZZ == 'A'
mask3 = wiktionary_lemma.member.str.contains('Ptz')
mask4 = wiktionary_lemma.data_id == suche_L_id
mask = mask1  &  mask2  &  mask3  &  mask4

suche_P_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

In [ ]:
# Das Ptz-Merkmal in wiktionary_lemma
mask1 = wiktionary_merkmal.data == suche_P
mask2 = wiktionary_merkmal.merkmal.str.startswith('Ptz')
mask3 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask4 = wiktionary_merkmal.lemma_id == suche_L_id   # zeigt aufs Verb
mask5 = wiktionary_merkmal.data_id  == suche_P_id   # zeigt aufs Adjektiv
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
wiktionary_merkmal[mask]

### Testlexeme

In [ ]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3 )
#grid(lemma_test)

In [ ]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

In [ ]:
# Fehler
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
maskC = lemma_test.check_member == ''
mask = maskA | maskB | maskC
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

In [ ]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

### lex_test

In [ ]:
# lex_test
from x32_Funktionen import *
lex_test = check_lex_test(lex_test, translate_lex)
#lex_test

In [ ]:
# Fehler
maskA = lex_test.lex_id == ''
maskB = lex_test.check_tag == ''
mask = maskA | maskB 
check_mask(lex_test,mask)

probleme = list(lex_test[mask].lemma)
lex_test[mask]

In [ ]:
# Sicherstellen: Keine Fehler
check_mask(lex_test,mask,0)

In [ ]:














bgc('green')

### Fehler analysieren

In [ ]:
sucheL = ['ausrutschen']
suche  = ['ausgerutscht']

In [ ]:
mask = lex_test.lex.isin(suche)
lex_test[mask]

In [ ]:
mask = translate_lex.data.isin(suche)
translate_lex[mask]

In [ ]:
mask = translate_lex_trash.data.isin(sucheL)
translate_lex_trash[mask]

In [ ]:
mask = wiktionary_lemma.lemma.isin( sucheL + suche )
wiktionary_lemma[mask]

In [ ]:
mask1 = wiktionary_merkmal.data.isin(suche)
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
wiktionary_merkmal[mask].sort_values(['lemma_id','merkmal','data'])

In [ ]:
grid(lex_test)

## Speichern ==========

In [ ]:
assert gründlich

In [ ]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,         wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_merkmal,       wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,  wiktionary_merkmal_text_filename_save)  
pak.dump_pickle(translate_lemma,          translate_lemma_filename_save) 
pak.dump_pickle(translate_lex,            translate_lex_filename_save)
pak.dump_pickle(translate_lex_trash,      translate_lex_trash_filename_save)
os.utime(verzeichnis_save)

In [ ]:
# drop_cols
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, ['data_id_old','data_id_bak'])

In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [ ]:
raise Stop

## suche_error ==========

In [ ]:
suche_debug = [suche_error[1]]
suche_debug

In [ ]:
# suche_debug wiktionary_lemma
mask = wiktionary_lemma.lemma.isin(suche_debug)
(wiktionary_lemma[mask])

In [ ]:
# suche_debug wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug)
(wiktionary_lemma_trash[mask])

In [ ]:
# translate_lex
search_str(translate_lex, suche_debug)

In [ ]:
# suche_debug translate_lemma
search_str(translate_lemma, suche_debug)

In [ ]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma.isin(suche_debug)
grid(wiktionary_merkmal,mask)

In [ ]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(['besonne'])
grid(wiktionary_merkmal,mask)

In [ ]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

In [ ]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

In [ ]:
# suche_debug wiktionary_merkmal_text
mask = wiktionary_merkmal.lemma_id.isin(suche_debug)
ids = set(wiktionary_merkmal[mask].lemma_id)
mask = wiktionary_merkmal_text.lemma_id.isin(ids)
grid(wiktionary_merkmal_text,mask)

In [ ]:
raise Stop

## Fragmente

In [ ]:
raise Stop

In [ ]:
# rename_col 
translate_lex = pak.rename_col(translate_lex, 'data_id',    'lex_id')
translate_lex = pak.rename_col(translate_lex, 'data_home',  'lex_home')
translate_lex = pak.rename_col(translate_lex, 'data',       'lex')
translate_lex = pak.rename_col(translate_lex, 'data_score', 'lex_score')

translate_lex_full = pak.rename_col(translate_lex_full, 'data_id',    'lex_id')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_home',  'lex_home')
translate_lex_full = pak.rename_col(translate_lex_full, 'data',       'lex')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_score', 'lex_score')

translate_lemma = pak.rename_col(translate_lemma, 'same_lexlemma', 'same_lemlemma')

In [ ]:
# rank!
#translate_lex = pak.rank(translate_lex, col_score='score', cols_group='data_id', on_conflict='first')

In [ ]:
# rank!
# Ein data_id pro data und lemma_tagZZ.
vorher = translate_lex.shape[0]
translate_lex = pak.rank(translate_lex, col_score='score', cols_group=['data','lemma_tagZZ'], on_conflict='first')
print(vorher - translate_lex.shape[0], 'Datensätze gelöscht')

In [ ]:
# Dups in data + lemma: Sind stets verschiedene lemma_tagZZ
mask = translate_lex.duplicated(subset=['data','lemma'], keep=False)
a = translate_lex[mask].sort_values('data_id')
#grid(a)

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id',  'data_id2', 'data_id_bak', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ 'max',      'min',      'max',         '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         'first',     'first',       'first',          'sum',   ]   
col_names   =  [ 'data_id',  'data_id2', 'data_id_bak', '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)